# Introduction
## This file starts with the data frame, which contains dyads built from the dyads_10k_embeddings. Then, we use both the analyst-by-analyst dataset and the stat summary, which contains monthly reports of analysts' consensus recommendations. We merged the recommendations with the filing files to find out and calculate the extent of positive evaluation of peer firms or rivals. 

## In the past, files for calculating variables at the NAICS4 and NAICS6 levels did not exclude firms within NAICS codes 22 and 52, as well as firms with no analyst coverage. In this file, we address that issue. Since we calculate industry-level variables at the NAICS4 or NAICS6 level, we do not need to exclude firms from NAICS 22 and 52, as we merge based on NAICS4 or NAICS6.

In [177]:
import numpy as np
import pandas as pd
embed_cols = ['embedding_' + str(i) for i in range(768)]

# See all the columns
pd.set_option('display.max_columns', None)

## The following data frames data and data_25 are the output of the dyad file ("creating the dyads").

In [132]:
data= pd.read_csv('/.../main_dataframe_10R_20240514.csv')
data_25 = pd.read_csv('/.../main_dataframe_25R_20240514.csv')
wrds_index= pd.read_csv('/.../wrds_index.csv')
wrds_vector_filter= pd.read_csv('/.../wrds_vector_filter.csv')

# Remove firms that have more than one filing report in a given year
wrds_vector_filter= wrds_vector_filter.drop_duplicates(subset=['GVKEY', 'year_modified'], keep='first')

In [179]:
print (data.shape[0],
data_25.shape[0])

348520 871300


In [133]:
data= data.rename(columns={'datadateF': 'datadate focal'})

data_25= data_25.rename(columns={'datadateF': 'datadate focal'})

data= data.rename(columns={'fdate': 'fdate focal'})

data_25= data_25.rename(columns={'fdate': 'fdate focal'})

wrds_index['datadate'] = pd.to_datetime(wrds_index['datadate'], format='%m/%d/%y')

wrds_index['datadate focal']= wrds_index ['datadate']

wrds_index['datadate peer']= wrds_index ['datadate']

wrds_index ['GVKEY focal']= wrds_index['GVKEY']

wrds_index ['GVKEY focal']= wrds_index['GVKEY']

wrds_vector_filter ['datadate']= pd.to_datetime(wrds_vector_filter ['datadate'])

data ['datadate focal']= pd.to_datetime(data ['datadate focal'])

data ['fdate focal']= pd.to_datetime(data ['fdate focal'])

wrds_vector_filter= pd.merge (wrds_vector_filter, wrds_index[['naicsh6','naicsh4', 'naicsh2','GVKEY', 'datadate']], on= ['GVKEY', 'datadate'], how= 'left' )

# Identifications at t1 for both a focal firm and the rival

In [181]:
## Focal firm 
data = pd.merge(data, wrds_vector_filter [['naicsh6','naicsh4', 'naicsh2','GVKEY', 'datadate', 'cusip_8']], 
                left_on= ['GVKEY focal', 'datadate focal'], 
                right_on=['GVKEY', 'datadate'], 
                how= 'left')

data.drop(['GVKEY', 'datadate'], axis=1, inplace=True)

data= data.rename(columns={'naicsh6': 'naicsh6 focal',
                          'naicsh4' : 'naicsh4 focal',
                          'naicsh2' : 'naicsh2 focal',
                          'cusip_8' : 'CUSIP focal'})

## Peer firm

data = pd.merge(data, wrds_vector_filter [['naicsh6','naicsh4', 'naicsh2','GVKEY', 'year_modified', 'cusip_8']], 
                left_on= ['GVKEY peer', 'year_modified'], 
                right_on=['GVKEY', 'year_modified'], 
                how= 'left')

data.drop(['GVKEY'], axis=1, inplace=True)

data= data.rename(columns={'naicsh6': 'naicsh6 peer',
                          'naicsh4' : 'naicsh4 peer',
                          'naicsh2' : 'naicsh2 peer',
                          'cusip_8' : 'CUSIP peer'})

In [182]:
# fdate peer 
data = pd.merge(data, wrds_vector_filter [['GVKEY', 'year_modified', 'fdate']], 
                left_on= ['GVKEY peer', 'year_modified'], 
                right_on=['GVKEY', 'year_modified'], 
                how= 'left')

data.drop(['GVKEY'], axis=1, inplace=True)


data= data.rename(columns={'fdate': 'fdate peer'})


# fdate peer for the file with 25 rivals

data_25 = pd.merge(data_25, wrds_vector_filter [['GVKEY', 'year_modified', 'fdate']], 
                left_on= ['GVKEY peer', 'year_modified'], 
                right_on=['GVKEY', 'year_modified'], 
                how= 'left')

data_25.drop(['GVKEY'], axis=1, inplace=True)

data_25= data_25.rename(columns={'fdate': 'fdate peer'})


# datadate peer 

data = pd.merge(data, wrds_vector_filter [['GVKEY', 'year_modified', 'datadate']], 
                left_on= ['GVKEY peer', 'year_modified'], 
                right_on=['GVKEY', 'year_modified'], 
                how= 'left')

data.drop(['GVKEY'], axis=1, inplace=True)

data= data.rename(columns={'datadate': 'datadate peer'})


# datadate peer for the file with 25 rivals

data_25 = pd.merge(data_25, wrds_vector_filter [['GVKEY', 'year_modified', 'datadate']], 
                left_on= ['GVKEY peer', 'year_modified'], 
                right_on=['GVKEY', 'year_modified'], 
                how= 'left')

data_25.drop(['GVKEY'], axis=1, inplace=True)

data_25= data_25.rename(columns={'datadate': 'datadate peer'})


In [183]:
#CUSIP codes have 8 characters, starting with zero if their char size is not 8. 
data['CUSIP focal'] = data['CUSIP focal'].astype(str).str.zfill(8)
data['CUSIP peer'] = data['CUSIP peer'].astype(str).str.zfill(8)

In [184]:
print(data.shape[0])
data.iloc[0:1]

348520


,year_modified,datadate focal,fdate focal,GVKEY focal,GVKEY peer,Cos_f1_p0,Cos_f1_p1,Cos_f0_p0,Cos_f0_p1,Cos_f2_p1,Cos_f2_p2,Cos_f1_p2,Cos_f0_p2,Cos_f4_p2,Cos_f3_p1,naicsh6 focal,naicsh4 focal,naicsh2 focal,CUSIP focal,naicsh6 peer,naicsh4 peer,naicsh2 peer,CUSIP peer,fdate peer,datadate peer
0,1994,1995-01-31,1995-04-07,1240,14832,0.856721,0.7788,0.861593,0.78367,0.791012,0.839087,0.824338,0.822772,0.839699,0.769806,445110,4451,44,01310410,446110,4461,44,26217510,1995-05-26,1995-02-28


-----
---

# Section 2:
## The input of section 2 was created in section 1

In [185]:
wrds_vector_filter ['cusip_8'] = wrds_vector_filter ['cusip_8'].astype(str).str.zfill(8)

wrds_vector_filter ['CUSIP peer'] = wrds_vector_filter ['cusip_8']

## Read the recommendation files

In [186]:
#Recommendation summary dataset
recomsu= pd.read_csv('/.../10K embeddings Dataset/Recomsum.csv')

# Reverse the recommendations numbers
recomsu['MEANREC']= 6- recomsu['MEANREC']

recomsu['MEDREC']= 6- recomsu['MEDREC']

# Rename for merging
recomsu.rename(columns={'CUSIP': 'CUSIP peer'}, inplace=True)

#Fix issues regarding CUSIP codes

wrds_vector_filter['CUSIP peer'] = wrds_vector_filter['CUSIP peer'].str.strip()

wrds_vector_filter['CUSIP peer'] = wrds_vector_filter['CUSIP peer'].str.upper()

wrds_vector_filter['CUSIP peer'] = wrds_vector_filter['CUSIP peer'].astype(str)


#Fix issues regarding CUSIP codes of recomsu file

recomsu['CUSIP peer'] = recomsu['CUSIP peer'].str.strip()
recomsu['CUSIP peer'] = recomsu['CUSIP peer'].str.upper()
recomsu['CUSIP peer'] = recomsu['CUSIP peer'].astype(str)

data['CUSIP focal']= data['CUSIP focal'].str.strip()
data['CUSIP focal']= data['CUSIP focal'].str.upper()
data['CUSIP focal']= data['CUSIP focal'].astype(str)


data['CUSIP peer']= data['CUSIP peer'].str.strip()
data['CUSIP peer']= data['CUSIP peer'].str.upper()
data['CUSIP peer']= data['CUSIP peer'].astype(str)


In [187]:
# Add the focal firm CUSIP

recomsu ['CUSIP focal']= recomsu ['CUSIP peer']

In [188]:
recomsu[recomsu['CUSIP peer'].apply(len) != 8]['CUSIP peer'].value_counts()

CUSIP peer
nan    1428
Name: count, dtype: int64

In [189]:
# We merged with the 'left' method, meaning that if the data of recommendations are not available, we will not have the data on the dependent variable. 

rec_sum= pd.merge(recomsu, wrds_vector_filter [['CUSIP peer','fdate','datadate','GVKEY']], on='CUSIP peer', how= 'left' )

## * rec_sum is merged with wrds_vector file including fdate, datadate, and GVKEY. However, the recomsu is the stat summary file, including only the monthly recommendations summary for each firm.

In [190]:

rec_sum['fdate peer']= rec_sum['fdate']

rec_sum['fdate peer'] = pd.to_datetime(rec_sum['fdate peer'])

rec_sum['STATPERS'] = pd.to_datetime(rec_sum['STATPERS'])

rec_sum['days_difference'] = (rec_sum['STATPERS'] - rec_sum['fdate peer']).dt.days

# For the monthly averages file
recomsu['STATPERS'] = pd.to_datetime(recomsu['STATPERS'])

recomsu['month_statp']= recomsu['STATPERS'].dt.month

recomsu['day_statp']= recomsu['STATPERS'].dt.day

# Mean recommendations calculation: *(Using StatSummary File)*
## The first method is in the next section by using 'apply' and writing functions

In [191]:
# Filtering those recommendations within a period of time before and after the filing date 
# days_difference = statperiod - fdate 
# rec_sum['days_difference'] = (rec_sum['STATPERS'] - rec_sum['fdate peer']).dt.days
# positive for after the filing/ negative for before the filing

rec_sum ['after_filing_90'] = np.where((rec_sum['days_difference'] >= 0) & (rec_sum['days_difference'] <= 90), 1, 0)

rec_sum ['before_filing_90'] = np.where((rec_sum['days_difference'] >= -90) & (rec_sum['days_difference'] < 0), 1, 0)

rec_sum ['after_filing_30'] = np.where((rec_sum['days_difference'] >= 0) & (rec_sum['days_difference'] <= 30), 1, 0)

rec_sum ['before_filing_30'] = np.where((rec_sum['days_difference'] >= -30) & (rec_sum['days_difference'] < 0), 1, 0)

rec_sum ['after_filing_60'] = np.where((rec_sum['days_difference'] >= 0) & (rec_sum['days_difference'] <= 60), 1, 0)

rec_sum ['before_filing_60'] = np.where((rec_sum['days_difference'] >= -60) & (rec_sum['days_difference'] < 0), 1, 0)

# 30, 60, 90 days after the previous filing date (30 days after t0):

rec_sum ['after_previous_filing_90'] = np.where((rec_sum['days_difference'] >= -360) & (rec_sum['days_difference'] <= -270), 1, 0)

rec_sum ['after_previous_filing_30'] = np.where((rec_sum['days_difference'] >= -360) & (rec_sum['days_difference'] <= -330), 1, 0)

rec_sum ['after_previous_filing_60'] = np.where((rec_sum['days_difference'] >= -360) & (rec_sum['days_difference'] <= -300), 1, 0)

rec_sum ['before_filing_360'] = np.where((rec_sum['days_difference'] >= -360) & (rec_sum['days_difference'] < 0), 1, 0)

rec_sum ['after_filing_180'] = np.where((rec_sum['days_difference'] >= 0) & (rec_sum['days_difference'] <= 180), 1, 0)

rec_sum ['after_filing_360'] = np.where((rec_sum['days_difference'] >= 0) & (rec_sum['days_difference'] <= 360), 1, 0)


# According to our new notations, 30 days after t-1, the whole t-1
rec_sum ['after_previous_2year_30'] = np.where((rec_sum['days_difference'] >= -720) & (rec_sum['days_difference'] < -690), 1, 0)

rec_sum ['previous_2year'] = np.where((rec_sum['days_difference'] >= -720) & (rec_sum['days_difference'] < -360), 1, 0)

# 30 and 60 days after t2
rec_sum ['nextyear_30'] = np.where((rec_sum['days_difference'] >= 360) & (rec_sum['days_difference'] < 390 ), 1, 0)

rec_sum ['nextyear_60'] = np.where((rec_sum['days_difference'] >= 360) & (rec_sum['days_difference'] < 420 ), 1, 0)



In [192]:
# Filter the rec_sum 

rec_sum= rec_sum[ (rec_sum['after_filing_360'] == 1) | (rec_sum['before_filing_360'] == 1) |  (rec_sum ['previous_2year'] ==1) |(rec_sum ['nextyear_60'] ==1) ]

rec_sum.shape[0]

2324456

## Weighted mean for the mean recommendation within a period:

In [193]:
def weighted_mean(df, condition_col):
    subset = df[df[condition_col] == 1]
    grouped = subset.groupby(['CUSIP peer', 'fdate peer']).apply(lambda x: pd.Series({
        'weighted_sum': (x['MEANREC'] * x['NUMREC']).sum(),
        'total_rec': x['NUMREC'].sum()
    })).reset_index()
    grouped[f'{condition_col}_w'] = grouped['weighted_sum'] / grouped['total_rec']
    return grouped[['CUSIP peer', 'fdate peer', f'{condition_col}_w']]


In [194]:
# Calculate the weighted mean recommendations

meanrec_after_90_w = weighted_mean(rec_sum, 'after_filing_90')
meanrec_before_90_w = weighted_mean(rec_sum, 'before_filing_90')
meanrec_after_30_w = weighted_mean(rec_sum, 'after_filing_30')
meanrec_before_30_w = weighted_mean(rec_sum, 'before_filing_30')
meanrec_after_60_w = weighted_mean(rec_sum, 'after_filing_60')
meanrec_before_60_w = weighted_mean(rec_sum, 'before_filing_60')
meanrec_previous_after_90_w = weighted_mean(rec_sum, 'after_previous_filing_90')
meanrec_previous_after_60_w = weighted_mean(rec_sum, 'after_previous_filing_60')
meanrec_previous_after_30_w = weighted_mean(rec_sum, 'after_previous_filing_30')
meanrec_before_360_w = weighted_mean(rec_sum, 'before_filing_360')
meanrec_after_previous_2year_30_w=  weighted_mean(rec_sum, 'after_previous_2year_30')
meanrec_previous_2year_w = weighted_mean(rec_sum, 'previous_2year')
meanrec_nextyear_30_w = weighted_mean(rec_sum, 'nextyear_30')
meanrec_nextyear_60_w = weighted_mean(rec_sum, 'nextyear_60')


/var/folders/8d/hsy71hlj20q70n8qz1r3s4s00000gn/T/ipykernel_30107/125858860.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = subset.groupby(['CUSIP peer', 'fdate peer']).apply(lambda x: pd.Series({
/var/folders/8d/hsy71hlj20q70n8qz1r3s4s00000gn/T/ipykernel_30107/125858860.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = subset.groupby(['CUSIP peer', 'fdate peer']).apply(lambda x: pd

## Non-weighted mean:

In [195]:
# meanrec_previous_after_90= rec_sum[rec_sum['after_previous_filing_90'] ==1 ].groupby(['CUSIP peer', 'fdate peer'])
# ['MEANREC'].mean().reset_index(name='meanrec_previous_after_90')
#‌ meanrec_after_90= rec_sum[rec_sum['after_filing_90'] ==1 ].groupby(['CUSIP peer', 'fdate peer'])['MEANREC'].mean().reset_index(name='meanrec_after_90')

meanrec_after_90 = rec_sum[rec_sum['after_filing_90'] ==1 ].groupby(['CUSIP peer', 'fdate peer'])['MEANREC'].mean().reset_index(name='after_filing_90')
meanrec_before_90 = rec_sum[rec_sum['before_filing_90'] ==1 ].groupby(['CUSIP peer', 'fdate peer'])['MEANREC'].mean().reset_index(name='before_filing_90')
meanrec_after_30 = rec_sum[rec_sum['after_filing_30'] ==1 ].groupby(['CUSIP peer', 'fdate peer'])['MEANREC'].mean().reset_index(name='after_filing_30')
meanrec_before_30 = rec_sum[rec_sum['before_filing_30'] ==1 ].groupby(['CUSIP peer', 'fdate peer'])['MEANREC'].mean().reset_index(name='before_filing_30')
meanrec_after_60 = rec_sum[rec_sum['after_filing_60'] ==1 ].groupby(['CUSIP peer', 'fdate peer'])['MEANREC'].mean().reset_index(name='after_filing_60')
meanrec_before_60 = rec_sum[rec_sum['before_filing_60'] ==1 ].groupby(['CUSIP peer', 'fdate peer'])['MEANREC'].mean().reset_index(name='before_filing_60')
meanrec_previous_after_90 = rec_sum[rec_sum['after_previous_filing_90'] ==1 ].groupby(['CUSIP peer', 'fdate peer'])['MEANREC'].mean().reset_index(name='after_previous_filing_90')
meanrec_previous_after_60 = rec_sum[rec_sum['after_previous_filing_60'] ==1 ].groupby(['CUSIP peer', 'fdate peer'])['MEANREC'].mean().reset_index(name='after_previous_filing_60')
meanrec_previous_after_30 = rec_sum[rec_sum['after_previous_filing_30'] ==1 ].groupby(['CUSIP peer', 'fdate peer'])['MEANREC'].mean().reset_index(name='after_previous_filing_30')
meanrec_before_360 = rec_sum[rec_sum['before_filing_360'] ==1 ].groupby(['CUSIP peer', 'fdate peer'])['MEANREC'].mean().reset_index(name='before_filing_360')
meanrec_after_previous_2year_30 = rec_sum[rec_sum['after_previous_2year_30'] ==1 ].groupby(['CUSIP peer', 'fdate peer'])['MEANREC'].mean().reset_index(name='after_previous_2year_30')
meanrec_previous_2year = rec_sum[rec_sum['previous_2year'] ==1 ].groupby(['CUSIP peer', 'fdate peer'])['MEANREC'].mean().reset_index(name='previous_2year')
meanrec_nextyear_30 = rec_sum[rec_sum['nextyear_30'] ==1 ].groupby(['CUSIP peer', 'fdate peer'])['MEANREC'].mean().reset_index(name='nextyear_30')
meanrec_nextyear_60 = rec_sum[rec_sum['nextyear_60'] ==1 ].groupby(['CUSIP peer', 'fdate peer'])['MEANREC'].mean().reset_index(name='nextyear_60')


In [196]:
meanrec_all_w = meanrec_after_90_w.merge(meanrec_before_90_w, on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(meanrec_after_30_w,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(meanrec_before_30_w,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(meanrec_after_60_w,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(meanrec_before_60_w,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(meanrec_previous_after_90_w,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(meanrec_previous_after_60_w,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(meanrec_previous_after_30_w,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(meanrec_before_360_w,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(meanrec_after_previous_2year_30_w,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(meanrec_previous_2year_w,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(meanrec_nextyear_30_w,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(meanrec_nextyear_60_w,  on=['CUSIP peer', 'fdate peer'], how='outer')
               

In [197]:
meanrec_all = meanrec_after_90.merge(meanrec_before_90, on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(meanrec_after_30,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(meanrec_before_30,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(meanrec_after_60,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(meanrec_before_60,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(meanrec_previous_after_90,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(meanrec_previous_after_60,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(meanrec_previous_after_30,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(meanrec_before_360,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(meanrec_after_previous_2year_30,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(meanrec_previous_2year,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(meanrec_nextyear_30,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(meanrec_nextyear_60,  on=['CUSIP peer', 'fdate peer'], how='outer')

In [198]:
meanrec_all = pd.merge( meanrec_all, meanrec_all_w, on=['CUSIP peer', 'fdate peer'], how='outer')

In [199]:
meanrec_all.iloc[0:1]

,CUSIP peer,fdate peer,after_filing_90,before_filing_90,after_filing_30,before_filing_30,after_filing_60,before_filing_60,after_previous_filing_90,after_previous_filing_60,after_previous_filing_30,before_filing_360,after_previous_2year_30,previous_2year,nextyear_30,nextyear_60,after_filing_90_w,before_filing_90_w,after_filing_30_w,before_filing_30_w,after_filing_60_w,before_filing_60_w,after_previous_filing_90_w,after_previous_filing_60_w,after_previous_filing_30_w,before_filing_360_w,after_previous_2year_30_w,previous_2year_w,nextyear_30_w,nextyear_60_w
0,00030710,2015-03-11,4.67,4.67,4.67,4.67,4.67,4.67,NaN,NaN,NaN,4.67,NaN,NaN,4.2,4.2,4.67,4.67,4.67,4.67,4.67,4.67,NaN,NaN,NaN,4.67,NaN,NaN,4.2,4.2


## Creating upgrades and downgrades

In [200]:
up_after_90= rec_sum[rec_sum['after_filing_90'] ==1 ].groupby(['CUSIP peer', 'fdate peer'])['NUMUP'].sum().reset_index(name='up_after_90')
down_after_90= rec_sum[rec_sum['after_filing_90'] ==1 ].groupby(['CUSIP peer', 'fdate peer'])['NUMDOWN'].sum().reset_index(name='down_after_90')

up_after_60= rec_sum[rec_sum['after_filing_60'] ==1 ].groupby(['CUSIP peer', 'fdate peer'])['NUMUP'].sum().reset_index(name='up_after_60')
down_after_60= rec_sum[rec_sum['after_filing_60'] ==1 ].groupby(['CUSIP peer', 'fdate peer'])['NUMDOWN'].sum().reset_index(name='down_after_60')

up_after_30= rec_sum[rec_sum['after_filing_30'] ==1 ].groupby(['CUSIP peer', 'fdate peer'])['NUMUP'].sum().reset_index(name='up_after_30')
down_after_30= rec_sum[rec_sum['after_filing_30'] ==1 ].groupby(['CUSIP peer', 'fdate peer'])['NUMDOWN'].sum().reset_index(name='down_after_30')

up_after_180= rec_sum[rec_sum['after_filing_180'] ==1 ].groupby(['CUSIP peer', 'fdate peer'])['NUMUP'].sum().reset_index(name='up_after_180')
down_after_180= rec_sum[rec_sum['after_filing_180'] ==1 ].groupby(['CUSIP peer', 'fdate peer'])['NUMDOWN'].sum().reset_index(name='down_after_180')

up_after_360= rec_sum[rec_sum['after_filing_360'] ==1 ].groupby(['CUSIP peer', 'fdate peer'])['NUMUP'].sum().reset_index(name='up_after_360')
down_after_360 = rec_sum[rec_sum['after_filing_360'] ==1 ].groupby(['CUSIP peer', 'fdate peer'])['NUMDOWN'].sum().reset_index(name='down_after_360')


In [201]:
updowns= pd.merge(up_after_90, down_after_90, on=['CUSIP peer', 'fdate peer'], how= 'outer')\
            .merge(up_after_60,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(down_after_60,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(up_after_30,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(down_after_30,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(up_after_180,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(down_after_180,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(up_after_360,  on=['CUSIP peer', 'fdate peer'], how='outer')\
            .merge(down_after_360,  on=['CUSIP peer', 'fdate peer'], how='outer')


## updown is a dataset includes upgrades and downgrades, built from rec_sum

## meanrec_all contains the mean of 'mean recommendations' in different time periods around the filing date

In [202]:
data ['fdate peer']= pd.to_datetime(data ['fdate peer'])

In [203]:
# Merge with the main dataset for peer firm mean recommendations

data = pd.merge(data, meanrec_all, on=['CUSIP peer', 'fdate peer'], how= 'left')

data = pd.merge(data, updowns, on=['CUSIP peer', 'fdate peer'], how= 'left')

# main_merged_rec= main_merged_rec.dropna(subset=['meanrec_before_filing_360', 'meanrec_after_filing_90'])

In [204]:
print (data.shape[0])
data.iloc[0:1]

348520


,year_modified,datadate focal,fdate focal,GVKEY focal,GVKEY peer,Cos_f1_p0,Cos_f1_p1,Cos_f0_p0,Cos_f0_p1,Cos_f2_p1,Cos_f2_p2,Cos_f1_p2,Cos_f0_p2,Cos_f4_p2,Cos_f3_p1,naicsh6 focal,naicsh4 focal,naicsh2 focal,CUSIP focal,naicsh6 peer,naicsh4 peer,naicsh2 peer,CUSIP peer,fdate peer,datadate peer,after_filing_90,before_filing_90,after_filing_30,before_filing_30,after_filing_60,before_filing_60,after_previous_filing_90,after_previous_filing_60,after_previous_filing_30,before_filing_360,after_previous_2year_30,previous_2year,nextyear_30,nextyear_60,after_filing_90_w,before_filing_90_w,after_filing_30_w,before_filing_30_w,after_filing_60_w,before_filing_60_w,after_previous_filing_90_w,after_previous_filing_60_w,after_previous_filing_30_w,before_filing_360_w,after_previous_2year_30_w,previous_2year_w,nextyear_30_w,nextyear_60_w,up_after_90,down_after_90,up_after_60,down_after_60,up_after_30,down_after_30,up_after_180,down_after_180,up_after_360,down_after_360
0,1994,1995-01-31,1995-04-07,1240,14832,0.856721,0.7788,0.861593,0.78367,0.791012,0.839087,0.824338,0.822772,0.839699,0.769806,445110,4451,44,01310410,446110,4461,44,26217510,1995-05-26,1995-02-28,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,4.0,4.5,NaN,4.0,NaN,2.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,4.0,4.333333,NaN,4.0,NaN,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# Create the meanrec difference:

## Weighted:

In [205]:
data['meanrec_diff_90_w']= data['after_filing_90_w']- data['before_filing_90_w']
data['meanrec_diff_30_w']= data['after_filing_30_w']- data['before_filing_30_w']
data['meanrec_diff_60_w']= data['after_filing_60_w']- data['before_filing_60_w']
data['meanrec_diff_90_360_w']= data['after_filing_90_w']- data['before_filing_360_w']
data['meanrec_diff_30_360_w']= data['after_filing_30_w']- data['before_filing_360_w']
data['meanrec_diff_90_previous_w']= data['after_filing_90_w']- data['after_previous_filing_90_w']
data['meanrec_diff_60_previous_w']= data['after_filing_60_w']- data['after_previous_filing_60_w']
data['meanrec_diff_30_previous_w']= data['after_filing_30_w']- data['after_previous_filing_30_w']
data['meanrec_diff_next_30_w']= data['nextyear_30_w']- data['after_filing_30_w']
data['meanrec_diff_next_60_w']= data['nextyear_60_w']- data['after_filing_60_w']
data['meanrec_diff_before_previous_30_w']= data['after_previous_filing_30_w']- data['after_previous_2year_30_w']
data['meanrec_diff_before_previous_360_w']= data['after_previous_filing_30_w']- data['previous_2year_w']


## Non-Weighted:

In [206]:
data['meanrec_diff_90']= data['after_filing_90']- data['before_filing_90']
data['meanrec_diff_30']= data['after_filing_30']- data['before_filing_30']
data['meanrec_diff_60']= data['after_filing_60']- data['before_filing_60']
data['meanrec_diff_90_360']= data['after_filing_90']- data['before_filing_360']
data['meanrec_diff_30_360']= data['after_filing_30']- data['before_filing_360']
data['meanrec_diff_90_previous']= data['after_filing_90']- data['after_previous_filing_90']
data['meanrec_diff_60_previous']= data['after_filing_60']- data['after_previous_filing_60']
data['meanrec_diff_30_previous']= data['after_filing_30']- data['after_previous_filing_30']
data['meanrec_diff_next_30']= data['nextyear_30']- data['after_filing_30']
data['meanrec_diff_next_60']= data['nextyear_60']- data['after_filing_60']
data['meanrec_diff_before_previous_30']= data['after_previous_filing_30']- data['after_previous_2year_30']
data['meanrec_diff_before_previous_360']= data['after_previous_filing_30']- data['previous_2year']


# Calculate the mean recommendation differences for focal firms

In [207]:
# Some preparations:

meanrec_all['CUSIP focal']= meanrec_all['CUSIP peer']

meanrec_all['fdate focal']= meanrec_all['fdate peer']

meanrec_all.drop('CUSIP peer', axis=1, inplace=True)

meanrec_all.drop('fdate peer', axis=1, inplace=True)

meanrec_all ['fdate focal']= pd.to_datetime(meanrec_all['fdate focal'])

data['fdate focal']= pd.to_datetime(data['fdate focal'])

# Focal firm mean recommendations

In [208]:
data = pd.merge(data, meanrec_all, on=['CUSIP focal', 'fdate focal'], how= 'left')

In [209]:
print(data.shape[0])
data.iloc[0:1]

348520


,year_modified,datadate focal,fdate focal,GVKEY focal,GVKEY peer,Cos_f1_p0,Cos_f1_p1,Cos_f0_p0,Cos_f0_p1,Cos_f2_p1,Cos_f2_p2,Cos_f1_p2,Cos_f0_p2,Cos_f4_p2,Cos_f3_p1,naicsh6 focal,naicsh4 focal,naicsh2 focal,CUSIP focal,naicsh6 peer,naicsh4 peer,naicsh2 peer,CUSIP peer,fdate peer,datadate peer,after_filing_90_x,before_filing_90_x,after_filing_30_x,before_filing_30_x,after_filing_60_x,before_filing_60_x,after_previous_filing_90_x,after_previous_filing_60_x,after_previous_filing_30_x,before_filing_360_x,after_previous_2year_30_x,previous_2year_x,nextyear_30_x,nextyear_60_x,after_filing_90_w_x,before_filing_90_w_x,after_filing_30_w_x,before_filing_30_w_x,after_filing_60_w_x,before_filing_60_w_x,after_previous_filing_90_w_x,after_previous_filing_60_w_x,after_previous_filing_30_w_x,before_filing_360_w_x,after_previous_2year_30_w_x,previous_2year_w_x,nextyear_30_w_x,nextyear_60_w_x,up_after_90,down_after_90,up_after_60,down_after_60,up_after_30,down_after_30,up_after_180,down_after_180,up_after_360,down_after_360,meanrec_diff_90_w,meanrec_diff_30_w,meanrec_diff_60_w,meanrec_diff_90_360_w,meanrec_diff_30_360_w,meanrec_diff_90_previous_w,meanrec_diff_60_previous_w,meanrec_diff_30_previous_w,meanrec_diff_next_30_w,meanrec_diff_next_60_w,meanrec_diff_before_previous_30_w,meanrec_diff_before_previous_360_w,meanrec_diff_90,meanrec_diff_30,meanrec_diff_60,meanrec_diff_90_360,meanrec_diff_30_360,meanrec_diff_90_previous,meanrec_diff_60_previous,meanrec_diff_30_previous,meanrec_diff_next_30,meanrec_diff_next_60,meanrec_diff_before_previous_30,meanrec_diff_before_previous_360,after_filing_90_y,before_filing_90_y,after_filing_30_y,before_filing_30_y,after_filing_60_y,before_filing_60_y,after_previous_filing_90_y,after_previous_filing_60_y,after_previous_filing_30_y,before_filing_360_y,after_previous_2year_30_y,previous_2year_y,nextyear_30_y,nextyear_60_y,after_filing_90_w_y,before_filing_90_w_y,after_filing_30_w_y,before_filing_30_w_y,after_filing_60_w_y,before_filing_60_w_y,after_previous_filing_90_w_y,after_previous_filing_60_w_y,after_previous_filing_30_w_y,before_filing_360_w_y,after_previous_2year_30_w_y,previous_2year_w_y,nextyear_30_w_y,nextyear_60_w_y
0,1994,1995-01-31,1995-04-07,1240,14832,0.856721,0.7788,0.861593,0.78367,0.791012,0.839087,0.824338,0.822772,0.839699,0.769806,445110,4451,44,01310410,446110,4461,44,26217510,1995-05-26,1995-02-28,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,4.0,4.5,NaN,4.0,NaN,2.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,4.0,4.333333,NaN,4.0,NaN,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.666667,0.666667,1.0,1.0,1.0,NaN,-3.0,NaN,0.0,0.0,0.0,0.0,0.5,0.5,1.0,1.0,1.0,NaN,-3.0,NaN,0.0,3.543333,3.63,3.55,3.61,3.565,3.61,3.776667,3.77,3.76,3.73,NaN,3.88,3.59,3.59,3.542712,3.63,3.55,3.61,3.564615,3.61,3.777222,3.770286,3.76,3.730856,NaN,3.88,3.59,3.59


In [210]:
data.rename(columns={'before_filing_360_x': 'before_filing_360_peer', 
                                'before_filing_360_y': 'before_filing_360_focal', 
                               'after_filing_90_x': 'after_filing_90_peer',
                               'after_filing_90_y':'after_filing_90_focal',
                               'before_filing_90_x' : 'before_filing_90_peer' ,
                               'before_filing_90_y': 'before_filing_90_focal',
                               'after_filing_30_x': 'after_filing_30_peer',
                               'after_filing_30_y': 'after_filing_30_focal',
                                'before_filing_30_x': 'before_filing_30_peer',
                                'before_filing_30_y': 'before_filing_30_focal',
                                'after_filing_60_x': 'after_filing_60_peer',
                                'after_filing_60_y': 'after_filing_60_focal',
                                'before_filing_60_x': 'before_filing_60_peer',
                                'before_filing_60_y': 'before_filing_60_focal',
                                'after_previous_filing_90_x': 'after_previous_filing_90_peer',
                                'after_previous_filing_90_y': 'after_previous_filing_90_focal',
                                'after_previous_filing_60_x': 'after_previous_filing_60_peer',
                                'after_previous_filing_60_y': 'after_previous_filing_60_focal',
                                'after_previous_filing_30_x': 'after_previous_filing_30_peer',
                                'after_previous_filing_30_y': 'after_previous_filing_30_focal',
                                'after_previous_2year_30_x': 'after_previous_2year_30_peer',
                                'after_previous_2year_30_y': 'after_previous_2year_30_focal',
                                'previous_2year_x': 'previous_2year_peer',
                                'previous_2year_y': 'previous_2year_focal',
                                'nextyear_30_x': 'nextyear_30_peer',
                                'nextyear_30_y': 'nextyear_30_focal',
                                'nextyear_60_x': 'nextyear_60_peer',
                                'nextyear_60_y': 'nextyear_60_focal'}, inplace=True)

In [211]:
data.rename(columns={'before_filing_360_w_x': 'before_filing_360_w_peer', 
                                'before_filing_360_w_y': 'before_filing_360_w_focal', 
                               'after_filing_90_w_x': 'after_filing_90_w_peer',
                               'after_filing_90_w_y':'after_filing_90_w_focal',
                               'before_filing_90_w_x' : 'before_filing_90_w_peer' ,
                               'before_filing_90_w_y': 'before_filing_90_w_focal',
                               'after_filing_30_w_x': 'after_filing_30_w_peer',
                               'after_filing_30_w_y': 'after_filing_30_w_focal',
                                'before_filing_30_w_x': 'before_filing_30_w_peer',
                                'before_filing_30_w_y': 'before_filing_30_w_focal',
                                'after_filing_60_w_x': 'after_filing_60_w_peer',
                                'after_filing_60_w_y': 'after_filing_60_w_focal',
                                'before_filing_60_w_x': 'before_filing_60_w_peer',
                                'before_filing_60_w_y': 'before_filing_60_w_focal',
                                'after_previous_filing_90_w_x': 'after_previous_filing_90_w_peer',
                                'after_previous_filing_90_w_y': 'after_previous_filing_90_w_focal',
                                'after_previous_filing_60_w_x': 'after_previous_filing_60_w_peer',
                                'after_previous_filing_60_w_y': 'after_previous_filing_60_w_focal',
                                'after_previous_filing_30_w_x': 'after_previous_filing_30_w_peer',
                                'after_previous_filing_30_w_y': 'after_previous_filing_30_w_focal',
                                'after_previous_2year_30_w_x': 'after_previous_2year_30_w_peer',
                                'after_previous_2year_30_w_y': 'after_previous_2year_30_w_focal',
                                'previous_2year_w_x': 'previous_2year_w_peer',
                                'previous_2year_w_y': 'previous_2year_w_focal',
                                'nextyear_30_w_x': 'nextyear_30_w_peer',
                                'nextyear_30_w_y': 'nextyear_30_w_focal',
                                'nextyear_60_w_x': 'nextyear_60_w_peer',
                                'nextyear_60_w_y': 'nextyear_60_w_focal'}, inplace=True)

In [212]:
data['meanrec_diff_90_focal']= data['after_filing_90_focal']- data['before_filing_90_focal']
data['meanrec_diff_30_focal']= data['after_filing_30_focal']- data['before_filing_30_focal']
data['meanrec_diff_60_focal']= data['after_filing_60_focal']- data['before_filing_60_focal']
data['meanrec_diff_90_360_focal']= data['after_filing_90_focal']- data['before_filing_360_focal']
data['meanrec_diff_30_360_focal']= data['after_filing_30_focal']- data['before_filing_360_focal']
data['meanrec_diff_90_previous_focal']= data['after_filing_90_focal']- data['after_previous_filing_90_focal']
data['meanrec_diff_60_previous_focal']= data['after_filing_60_focal']- data['after_previous_filing_60_focal']
data['meanrec_diff_30_previous_focal']= data['after_filing_30_focal']- data['after_previous_filing_30_focal']

data['meanrec_diff_next_30_focal']= data['nextyear_30_focal']- data['after_filing_30_focal']
data['meanrec_diff_next_60_focal']= data['nextyear_60_focal']- data['after_filing_60_focal']
data['meanrec_diff_before_previous_30_focal']= data['after_previous_filing_30_focal']- data['after_previous_2year_30_focal']
data['meanrec_diff_before_previous_360_focal']= data['after_previous_filing_30_focal']- data['previous_2year_focal']


In [213]:
data['meanrec_diff_90_w_focal']= data['after_filing_90_w_focal']- data['before_filing_90_w_focal']
data['meanrec_diff_30_w_focal']= data['after_filing_30_w_focal']- data['before_filing_30_w_focal']
data['meanrec_diff_60_w_focal']= data['after_filing_60_w_focal']- data['before_filing_60_w_focal']
data['meanrec_diff_90_360_w_focal']= data['after_filing_90_w_focal']- data['before_filing_360_w_focal']
data['meanrec_diff_30_360_w_focal']= data['after_filing_30_w_focal']- data['before_filing_360_w_focal']
data['meanrec_diff_90_previous_w_focal']= data['after_filing_90_w_focal']- data['after_previous_filing_90_w_focal']
data['meanrec_diff_60_previous_w_focal']= data['after_filing_60_w_focal']- data['after_previous_filing_60_w_focal']
data['meanrec_diff_30_previous_w_focal']= data['after_filing_30_w_focal']- data['after_previous_filing_30_w_focal']

data['meanrec_diff_next_30_w_focal']= data['nextyear_30_w_focal']- data['after_filing_30_w_focal']
data['meanrec_diff_next_60_w_focal']= data['nextyear_60_w_focal']- data['after_filing_60_w_focal']
data['meanrec_diff_before_previous_30_w_focal']= data['after_previous_filing_30_w_focal']- data['after_previous_2year_30_w_focal']
data['meanrec_diff_before_previous_360_w_focal']= data['after_previous_filing_30_w_focal']- data['previous_2year_w_focal']


----

# Adding the Abnormal returns to the main_merged dataset (The continuation of stat summary section)

In [214]:
# Create the input for the creation of abnormal return

# wrds_vector_filter [['CUSIP peer', 'fdate']].to_csv('output.txt', index=False, sep=' ', header=False)

In [215]:
abn1= pd.read_csv('/.../abn1.csv')
abn3= pd.read_csv('/.../abn3.csv')
abn5= pd.read_csv('/.../abn5.csv')
abn7= pd.read_csv('/.../abn7.csv')

abn1= abn1.rename(columns= {'cusip': 'CUSIP peer', 'evtdate':'fdate peer','car':'car1' })
abn3= abn3.rename(columns= {'cusip': 'CUSIP peer', 'evtdate':'fdate peer','car':'car3' })
abn5= abn5.rename(columns= {'cusip': 'CUSIP peer', 'evtdate':'fdate peer','car':'car5'})
abn7= abn7.rename(columns= {'cusip': 'CUSIP peer', 'evtdate':'fdate peer','car':'car7'})


abn1['fdate peer']= pd.to_datetime(abn1['fdate peer'])
abn3['fdate peer']= pd.to_datetime(abn3['fdate peer'])
abn5['fdate peer']= pd.to_datetime(abn5['fdate peer'])
abn7['fdate peer']= pd.to_datetime(abn7['fdate peer'])


abn1= abn1[['CUSIP peer', 'fdate peer', 'car1']]
abn3= abn3[['CUSIP peer', 'fdate peer', 'car3']]
abn5= abn5[['CUSIP peer', 'fdate peer', 'car5']]
abn7= abn7[['CUSIP peer', 'fdate peer', 'car7']]

In [216]:
# This merged data contains all the abnormal returns calculating after the filing date (1-3-5-7)

abn_all = abn1.merge(abn3, on=['CUSIP peer', 'fdate peer'], how='outer')\
           .merge(abn5,  on=['CUSIP peer', 'fdate peer'], how='outer')\
           .merge(abn7,  on=['CUSIP peer', 'fdate peer'], how='outer')

In [217]:
#Merge the two datasets
data = pd.merge( data, abn_all, on=['CUSIP peer', 'fdate peer'], how='left')

In [218]:
# Remove NaN
data.dropna().shape[0]

# around 1000 missing abnormal returns

139882

---

# Reading the accounting data

"Various accounting variables are calculated based on the existing columns, such as number of employees, EPA, slack measures, R&D share of total expense, advertising share of total expense, recognized intangible assets as part of total assets, depreciation as part of total assets, total merger, and leverage. Additionally, the number of unique firms for each industry-year combination is calculated."

#### n_emp
#### EPS
#### slack_avail (slack measures)
#### rd_f (r&d share of total expense)
#### adv_f (advertising share of total expense)
#### intang_f (recognized intangible assets as part of total assets)
#### dpt_f (depreciation as part of total assets)
#### mergers (Total merger)
#### leverage 

In [219]:
def modify_year(row):
    if row['monthd'] < 7:
        return row['yeard'] - 1
    else:
        return row['yeard']
 
# We modify the year from the date that the report has been published. If it was in the 6 month, 
# We assume that most of the report was from the past year. However, after the month 6, we consider the same year. 

In [ ]:
compustat= pd.read_csv('/.../compustat.csv')

compustat['GVKEY peer']= compustat['GVKEY']
compustat['GVKEY focal']= compustat['GVKEY']

compustat['datadate peer']= compustat['datadate']
compustat['datadate focal']= compustat['datadate']


compustat['datadate']= pd.to_datetime(compustat['datadate'])
compustat['datadate peer']= pd.to_datetime(compustat['datadate peer'])
compustat['datadate focal']= pd.to_datetime(compustat['datadate focal'])

In [221]:
compustat['datadate']= pd.to_datetime(compustat['datadate'])

compustat['yeard']= compustat['datadate'].dt.year

compustat['monthd']= compustat['datadate'].dt.month

compustat ['year_modified'] = compustat.apply(modify_year, axis=1)

## ROE and ROA

In [222]:

#ROA and ROE
# 'at' is total asset

compustat['roa'] = np.where(compustat['at'] != 0, compustat['ni'] / compustat['at'], np.nan)

compustat['roe'] = np.where( compustat['csho'] * compustat['prcc_f'] !=0, compustat['ni']/(compustat['csho'] * compustat['prcc_f']), np.nan)

# Create the log of the variables

compustat['log_at'] = np.log(compustat['at']+1)

compustat['log_n_emp'] = np.log(compustat['n_emp']+1)


# Tobin's Q

compustat ['market_value_equity'] = compustat ['prcc_f'] * compustat ['csho']
compustat ['book_value_debt'] = compustat ['at'] - compustat ['ceq']
compustat ['market_value_assets'] = compustat ['market_value_equity'] + compustat['book_value_debt']
compustat['tobins_q'] = compustat['market_value_assets'] / compustat['at']


# R&D intensity

compustat['rd_intensity'] = np.where((compustat['sale'] != 0) & (np.isnan(compustat['xrd'])), 0,
                                     np.where(compustat['sale'] != 0, compustat['xrd'] / compustat['sale'], np.nan))


# ROS

compustat['ros'] = np.where(compustat['sale'] != 0, compustat['ni'] / compustat['sale'], np.nan)

In [223]:
compustat.rename(columns={'cusip': 'CUSIP'}, inplace=True)
compustat ['CUSIP'] = compustat['CUSIP'].str.strip()
compustat ['CUSIP'] = compustat['CUSIP'].str.upper()
compustat ['CUSIP'] = compustat['CUSIP'].astype(str)
compustat ['CUSIP'] = compustat ['CUSIP'].astype(str).str.zfill(8)

# Exclude firms with no analyst coverage

## Since we calculate variables at NAICS 4 and 6, excluding 22 and 52 from the Compustat dataset does not make any difference.

In [ ]:
eps_estimate= pd.read_csv('/.../EPS_estimate_1FPI.csv') # analyst-by-analyst forecast file
eps_estimate ['CUSIP'] = eps_estimate['CUSIP'].str.strip()
eps_estimate ['CUSIP'] = eps_estimate['CUSIP'].str.upper()
eps_estimate ['CUSIP'] = eps_estimate['CUSIP'].astype(str)

analysts_year_firm= eps_estimate.groupby(['CUSIP', 'FPEDATS'])['ANALYS'].unique().reset_index(name='analysts_year_firm')

analysts_year_firm['datadate']= analysts_year_firm['FPEDATS']

analysts_year_firm ['analysts_focal_number'] = analysts_year_firm ['analysts_year_firm'].apply(lambda x: len(x) if isinstance(x, (list, np.ndarray)) else 0)

analysts_year_firm_filtered = analysts_year_firm [analysts_year_firm['analysts_focal_number'] >= 1]

compustat['datadate'] = pd.to_datetime(compustat['datadate'])

analysts_year_firm_filtered['datadate'] = pd.to_datetime(analysts_year_firm_filtered['datadate'])

compustat['CUSIP'] = compustat['CUSIP'].apply(lambda x: x[:-1])

# Apply the conditions: 
compustat_nocoverage_removed = pd.merge(compustat, analysts_year_firm_filtered, on=['CUSIP', 'datadate'], how='inner')

## HHI for NAICS4 and NAICS6

In [225]:
# market share of each firm in their industry (naicsh6)

ind_col = 'naicsh6'

total_sales = compustat_nocoverage_removed.groupby(['year_modified', ind_col])['sale'].sum().reset_index(name=f'total_sales_{ind_col}')

compustat_nocoverage_removed = compustat_nocoverage_removed.merge(total_sales, on=['year_modified', ind_col])

compustat_nocoverage_removed[f'market_share_{ind_col}'] = compustat_nocoverage_removed['sale'] / compustat_nocoverage_removed[f'total_sales_{ind_col}']

# market share of each firm in their industry (naicsh4)

ind_col2 = 'naicsh4'

total_sales2 = compustat_nocoverage_removed.groupby(['year_modified', ind_col2])['sale'].sum().reset_index(name=f'total_sales_{ind_col2}')

compustat_nocoverage_removed = compustat_nocoverage_removed.merge(total_sales2, on=['year_modified', ind_col2])

compustat_nocoverage_removed[f'market_share_{ind_col2}'] = compustat_nocoverage_removed['sale'] / compustat_nocoverage_removed[f'total_sales_{ind_col2}']


# hhi 6, as an indication of industry concentration

compustat_nocoverage_removed[f'squared_market_share_{ind_col}'] = compustat_nocoverage_removed[f'market_share_{ind_col}'] ** 2
hhi = compustat_nocoverage_removed.groupby(['year_modified', ind_col])[f'squared_market_share_{ind_col}'].sum().reset_index(name=f'hhi_{ind_col}')
compustat_nocoverage_removed = compustat_nocoverage_removed.merge(hhi, on=['year_modified', ind_col])


# hhi 4, as an indication of industry concentration

compustat_nocoverage_removed[f'squared_market_share_{ind_col2}'] = compustat_nocoverage_removed[f'market_share_{ind_col2}'] ** 2
hhi2 = compustat_nocoverage_removed.groupby(['year_modified', ind_col2])[f'squared_market_share_{ind_col2}'].sum().reset_index(name=f'hhi_{ind_col2}')
compustat_nocoverage_removed = compustat_nocoverage_removed.merge(hhi2, on=['year_modified', ind_col2])

# 1 - hhi
compustat_nocoverage_removed['hhi_naicsh6_inv']= 1- compustat_nocoverage_removed['hhi_naicsh6']

compustat_nocoverage_removed['hhi_naicsh4_inv']= 1- compustat_nocoverage_removed['hhi_naicsh4']

In [226]:
compustat= pd.merge(compustat, compustat_nocoverage_removed[['GVKEY', 'datadate', 'hhi_naicsh6', 'hhi_naicsh4', 'hhi_naicsh6_inv', 'hhi_naicsh4_inv','market_share_naicsh4', 'market_share_naicsh6']], 
                    on=['GVKEY', 'datadate'], how='left')

### Industry growth for one year: (naicsh4)

In [227]:
industry_sales = compustat_nocoverage_removed.groupby(['naicsh4', 'year_modified'])['sale'].sum().reset_index()

industry_sales['year_diff'] = industry_sales.groupby('naicsh4')['year_modified'].diff()

industry_sales['sales_growth_naicsh4'] = industry_sales.groupby('naicsh4')['sale'].pct_change() * 100

# Set the sales growth to NaN wherever year_diff is not 1
industry_sales.loc[industry_sales['year_diff'] != 1, 'sales_growth_naicsh4'] = None

In [228]:
compustat = compustat.merge(industry_sales[['naicsh4', 'year_modified', 'sales_growth_naicsh4']], on=['year_modified', 'naicsh4'])

## Winsorize the variables

In [229]:
def winsorize(input_series, p):
    q_bottom = input_series.quantile(p/100)
    q_top = input_series.quantile(1-p/100)
    output_series = input_series.clip(lower=q_bottom, upper=q_top)
    return output_series


# Winsorizing accounting variables
winsorize_cols = ['EPS', 'ros', 'tobins_q', 'sale', 'slack_avail', 'rd_f', 'adv_f', 'intang_f', 'dpt_f', 'leverage', 'hhi_naicsh4', 
                  'hhi_naicsh6', 'sales_growth_naicsh4', 'rd_intensity', 'mergers', 'roa', 'roe', 'log_at', 'log_n_emp', 
                 'market_share_naicsh4', 'market_share_naicsh6', 'hhi_naicsh4_inv', 'hhi_naicsh6_inv' ]

winsorize_cols_nocoverage = ['EPS', 'ros', 'tobins_q', 'sale', 'slack_avail', 'rd_f', 'adv_f', 'intang_f', 'dpt_f', 'leverage', 'hhi_naicsh4', 
                  'hhi_naicsh6' , 'rd_intensity', 'mergers', 'roa', 'roe', 'log_at', 'log_n_emp', 
                 'market_share_naicsh4', 'market_share_naicsh6', 'hhi_naicsh4_inv', 'hhi_naicsh6_inv' ]
    
for col in winsorize_cols:
    compustat[col+'_win'] = winsorize(compustat[col], 1)


for col in winsorize_cols_nocoverage:
    compustat_nocoverage_removed[col+'_win'] = winsorize(compustat_nocoverage_removed[col], 1)


In [230]:
# filter
compustat_filtered= compustat[['GVKEY peer', 'GVKEY focal', 'datadate peer','datadate focal', 'n_emp','log_n_emp', 'hhi_naicsh6_inv','hhi_naicsh4_inv', 'hhi_naicsh6_inv_win', 
                               'hhi_naicsh4_inv_win', 
                               'roa', 'roa_win','roe','roe_win', 'sales_growth_naicsh4', 'sales_growth_naicsh4_win', 'rd_intensity', 'rd_intensity_win', 'naicsh4',
                               'sale', 'sale_win', 'slack_avail', 'slack_avail_win', 'adv_f', 'adv_f_win', 'intang_f', 'intang_f_win', 'dpt_f', 'dpt_f_win',
                               'leverage', 'leverage_win', 'tobins_q', 'tobins_q_win',  
                               'market_share_naicsh4', 'prcc_f', 'EPS','EPS_win','log_at_win', 'at', 'log_at','ni', 'rd_f', 'mergers','mergers_win', 'year_modified',
                               'market_share_naicsh6', 
                            'market_share_naicsh4_win', 'naicsh6', 
                             'ros_win', 'ros']] 


## Adding firms' accounting data to the main dataset:

In [231]:
# Focal firm
data_acc = pd.merge(data, compustat_filtered, on=['GVKEY focal', 'datadate focal'], how='left')
data_acc.drop(['GVKEY peer_y', 'datadate peer_y', 'year_modified_y'], axis=1, inplace=True)
data_acc = data_acc.rename(columns={'GVKEY peer_x': 'GVKEY peer', 'datadate peer_x': 'datadate peer'})
data_acc = data_acc.rename(columns={'year_modified_x': 'year_modified'})

# Peer firm
data_acc = pd.merge(data_acc, compustat_filtered, on=['GVKEY peer', 'datadate peer'], how='left')
data_acc.drop(['year_modified_y'], axis=1, inplace=True)
data_acc.drop(['GVKEY focal_y', 'datadate focal_y'], axis=1, inplace=True)

data_acc = data_acc.rename(columns={'GVKEY focal_x': 'GVKEY focal', 'datadate focal_x': 'datadate focal'})
data_acc = data_acc.rename(columns={'year_modified_x': 'year_modified'})


In [234]:
data_acc.shape[0]

348520

## Creating relative controls

In [236]:
# ROA
data_acc['roa_rel']= data_acc['roa_win_x'] - data_acc['roa_win_y']

# ROE
data_acc['roe_rel']= data_acc['roe_win_x'] - data_acc['roe_win_y']

# Total assets
data_acc['log_at_rel_win']=  data_acc['log_at_win_x'] - data_acc['log_at_win_y']

data_acc['log_at_rel']=  data_acc['log_at_x'] - data_acc['log_at_y']

# ROS
data_acc['ros_rel']= data_acc['ros_win_x'] - data_acc['ros_win_y']

# Normalized to the focal firm
data_acc['roe_rel_focal']=  np.where( data_acc['roe_win_x'] != 0, (data_acc['roe_win_x'] - data_acc['roe_win_y'])/ data_acc['roe_win_x'],np.nan)

data_acc['roa_rel_focal']=  np.where ( data_acc['roa_win_x'] != 0, (data_acc['roa_win_x'] - data_acc['roa_win_y'])/ data_acc['roa_win_x'], np.nan)

# Tobin's Q
data_acc['tobins_q_rel']= (data_acc['tobins_q_win_x'] - data_acc['tobins_q_win_y'])
data_acc['tobins_q_rel_focal']=  (data_acc['tobins_q_win_x'] - data_acc['tobins_q_win_y'])/ data_acc['tobins_q_win_x']

#EPS

data_acc['EPS_rel']= data_acc['EPS_win_x'] - data_acc['EPS_win_y']
data_acc['EPS_rel_focal']= np.where(data_acc['EPS_win_x'] !=0, (data_acc['EPS_win_x'] - data_acc['EPS_win_y'])/ data_acc['EPS_win_x'], np.nan)


-----

# Common analysts

In [ ]:
eps_estimate= pd.read_csv('/.../EPS_estimate_1FPI.csv') # analyst-by-analyst forecast file

eps_estimate['ANNDATS']= pd.to_datetime(eps_estimate['ANNDATS'])

eps_estimate['FPEDATS']= pd.to_datetime(eps_estimate['FPEDATS'])

eps_estimate= eps_estimate.rename(columns= {'CUSIP': 'CUSIP focal'})

eps_estimate['CUSIP peer']= eps_estimate['CUSIP focal']

data_acc['datadate focal']= pd.to_datetime(data_acc['datadate focal'])

data_acc['datadate peer']= pd.to_datetime(data_acc['datadate peer'])

analysts_year_firm= eps_estimate.groupby(['CUSIP focal', 'FPEDATS'])['ANALYS'].unique().reset_index(name='analysts_year_firm')

analysts_year_firm['CUSIP peer']= analysts_year_firm['CUSIP focal']

analysts_year_firm['datadate focal']= analysts_year_firm['FPEDATS']

analysts_year_firm['datadate peer']= analysts_year_firm['FPEDATS']

### Adding focal firm analysts

In [239]:
# Focal firm analysts

data_acc = pd.merge(data_acc, analysts_year_firm[['CUSIP focal', 'datadate focal', 'analysts_year_firm']], on=['CUSIP focal', 'datadate focal'], how= 'left')

data_acc = data_acc.rename(columns={'analysts_year_firm': 'analysts_focal'})

data_acc['analysts_focal_number'] = data_acc['analysts_focal'].apply(lambda x: len(x) if isinstance(x, (list, np.ndarray)) else 0)


# Peer firm analysts
data_acc = pd.merge(data_acc, analysts_year_firm[['CUSIP peer', 'datadate peer', 'analysts_year_firm']], on=['CUSIP peer', 'datadate peer'], how= 'left')

data_acc = data_acc.rename(columns={'analysts_year_firm': 'analysts_peer'})

data_acc['analysts_peer_number'] = data_acc['analysts_peer'].apply(lambda x: len(x) if isinstance(x, (list, np.ndarray)) else 0)

data_acc['analysts_focal_number'] = data_acc['analysts_focal'].apply(lambda x: len(x) if isinstance(x, (list, np.ndarray)) else 0)


In [240]:
# Common analysts number, using intersetion

data_acc['common_analysts_number'] = data_acc.apply(lambda x: len(np.intersect1d(x['analysts_focal'], x['analysts_peer'])) 
                                    if isinstance(x['analysts_focal'], np.ndarray) and isinstance(x['analysts_peer'], np.ndarray) 
                                    else 0, axis=1)

# Common analysts number, using union

data_acc['unique_analysts_number'] = data_acc['analysts_peer_number'] + data_acc['analysts_focal_number'] - data_acc['common_analysts_number']

## We create different forms of common analysts variable

In [241]:
data_acc['common_analysts_unique']= np.where(data_acc['unique_analysts_number'] != 0, 
                                                         data_acc['common_analysts_number']/ data_acc['unique_analysts_number'],0)

data_acc['common_analysts_mean']= np.where(data_acc['unique_analysts_number'] != 0, 
                                                         data_acc['common_analysts_number']/ (data_acc['analysts_peer_number'] + data_acc['analysts_focal_number']),0)

data_acc['common_analysts_focal']= np.where(data_acc['analysts_focal_number'] != 0, 
                                                         data_acc['common_analysts_number']/ data_acc['analysts_focal_number'] ,0)

data_acc['common_analysts_peer']= np.where(data_acc['analysts_focal_number'] != 0, 
                                                         data_acc['common_analysts_number']/ data_acc['analysts_peer_number'] ,0)

In [242]:
print(data_acc.shape[0])    
data_acc.iloc[0:1]

348520


,year_modified,datadate focal,fdate focal,GVKEY focal,GVKEY peer,Cos_f1_p0,Cos_f1_p1,Cos_f0_p0,Cos_f0_p1,Cos_f2_p1,Cos_f2_p2,Cos_f1_p2,Cos_f0_p2,Cos_f4_p2,Cos_f3_p1,naicsh6 focal,naicsh4 focal,naicsh2 focal,CUSIP focal,naicsh6 peer,naicsh4 peer,naicsh2 peer,CUSIP peer,fdate peer,datadate peer,after_filing_90_peer,before_filing_90_peer,after_filing_30_peer,before_filing_30_peer,after_filing_60_peer,before_filing_60_peer,after_previous_filing_90_peer,after_previous_filing_60_peer,after_previous_filing_30_peer,before_filing_360_peer,after_previous_2year_30_peer,previous_2year_peer,nextyear_30_peer,nextyear_60_peer,after_filing_90_w_peer,before_filing_90_w_peer,after_filing_30_w_peer,before_filing_30_w_peer,after_filing_60_w_peer,before_filing_60_w_peer,after_previous_filing_90_w_peer,after_previous_filing_60_w_peer,after_previous_filing_30_w_peer,before_filing_360_w_peer,after_previous_2year_30_w_peer,previous_2year_w_peer,nextyear_30_w_peer,nextyear_60_w_peer,up_after_90,down_after_90,up_after_60,down_after_60,up_after_30,down_after_30,up_after_180,down_after_180,up_after_360,down_after_360,meanrec_diff_90_w,meanrec_diff_30_w,meanrec_diff_60_w,meanrec_diff_90_360_w,meanrec_diff_30_360_w,meanrec_diff_90_previous_w,meanrec_diff_60_previous_w,meanrec_diff_30_previous_w,meanrec_diff_next_30_w,meanrec_diff_next_60_w,meanrec_diff_before_previous_30_w,meanrec_diff_before_previous_360_w,meanrec_diff_90,meanrec_diff_30,meanrec_diff_60,meanrec_diff_90_360,meanrec_diff_30_360,meanrec_diff_90_previous,meanrec_diff_60_previous,meanrec_diff_30_previous,meanrec_diff_next_30,meanrec_diff_next_60,meanrec_diff_before_previous_30,meanrec_diff_before_previous_360,after_filing_90_focal,before_filing_90_focal,after_filing_30_focal,before_filing_30_focal,after_filing_60_focal,before_filing_60_focal,after_previous_filing_90_focal,after_previous_filing_60_focal,after_previous_filing_30_focal,before_filing_360_focal,after_previous_2year_30_focal,previous_2year_focal,nextyear_30_focal,nextyear_60_focal,after_filing_90_w_focal,before_filing_90_w_focal,after_filing_30_w_focal,before_filing_30_w_focal,after_filing_60_w_focal,before_filing_60_w_focal,after_previous_filing_90_w_focal,after_previous_filing_60_w_focal,after_previous_filing_30_w_focal,before_filing_360_w_focal,after_previous_2year_30_w_focal,previous_2year_w_focal,nextyear_30_w_focal,nextyear_60_w_focal,meanrec_diff_90_focal,meanrec_diff_30_focal,meanrec_diff_60_focal,meanrec_diff_90_360_focal,meanrec_diff_30_360_focal,meanrec_diff_90_previous_focal,meanrec_diff_60_previous_focal,meanrec_diff_30_previous_focal,meanrec_diff_next_30_focal,meanrec_diff_next_60_focal,meanrec_diff_before_previous_30_focal,meanrec_diff_before_previous_360_focal,meanrec_diff_90_w_focal,meanrec_diff_30_w_focal,meanrec_diff_60_w_focal,meanrec_diff_90_360_w_focal,meanrec_diff_30_360_w_focal,meanrec_diff_90_previous_w_focal,meanrec_diff_60_previous_w_focal,meanrec_diff_30_previous_w_focal,meanrec_diff_next_30_w_focal,meanrec_diff_next_60_w_focal,meanrec_diff_before_previous_30_w_focal,meanrec_diff_before_previous_360_w_focal,car1,car3,car5,car7,n_emp_x,log_n_emp_x,hhi_naicsh6_inv_x,hhi_naicsh4_inv_x,hhi_naicsh6_inv_win_x,hhi_naicsh4_inv_win_x,roa_x,roa_win_x,roe_x,roe_win_x,sales_growth_naicsh4_x,sales_growth_naicsh4_win_x,rd_intensity_x,rd_intensity_win_x,naicsh4_x,sale_x,sale_win_x,slack_avail_x,slack_avail_win_x,adv_f_x,adv_f_win_x,intang_f_x,intang_f_win_x,dpt_f_x,dpt_f_win_x,leverage_x,leverage_win_x,tobins_q_x,tobins_q_win_x,market_share_naicsh4_x,prcc_f_x,EPS_x,EPS_win_x,log_at_win_x,at_x,log_at_x,ni_x,rd_f_x,mergers_x,mergers_win_x,market_share_naicsh6_x,market_share_naicsh4_win_x,naicsh6_x,ros_win_x,ros_x,n_emp_y,log_n_emp_y,hhi_naicsh6_inv_y,hhi_naicsh4_inv_y,hhi_naicsh6_inv_win_y,hhi_naicsh4_inv_win_y,roa_y,roa_win_y,roe_y,roe_win_y,sales_growth_naicsh4_y,sales_growth_naicsh4_win_y,rd_intensity_y,rd_intensity_win_y,naicsh4_y,sale_y,sale_win_y,slack_avail_y,slack_avail_win_y,adv_f_y,adv_f_win_y,intang_f_y,intang_f

## Dummy variables for indicating the extent of move similarity: focal firm toward the peer [t0, t+1] and peer toward the focal firm [t-1, t0]

In [244]:
# Variable for indicating whether the focal firm moves toward the peer firm's position (from t0 to t1)
data_acc['focal_move_similarity']= data_acc['Cos_f2_p1'] - data_acc['Cos_f1_p1']

# Variable for indicating whether the peer firm's move is moving close or getting away (from t-1 to t0)
data_acc['peer_move_similarity']= data_acc['Cos_f0_p1'] - data_acc['Cos_f0_p0']


data_acc['focal_move_similarity_lagged']= data_acc['Cos_f1_p0'] - data_acc['Cos_f0_p0']

data_acc['focal_move_similarity_d'] = np.where(data_acc['focal_move_similarity'] > 0, 1, 0)

data_acc['peer_move_similarity_d'] = np.where(data_acc['peer_move_similarity'] > 0, 1, 0)

data_acc['focal_move_similarity_lagged_d'] = np.where(data_acc['focal_move_similarity_lagged'] > 0, 1, 0)

data_acc.rename(columns={'peer_move_similarity_d': 'move_toward'}, inplace=True)

data_acc['move_away'] = np.where(data_acc['peer_move_similarity'] < 0, 1, 0)

# Robustness

data_acc['focal_move_similarity_t3']= data_acc['Cos_f3_p1'] - data_acc['Cos_f1_p1']

data_acc['peer_move_similarity_t2']= data_acc['Cos_f0_p2'] - data_acc['Cos_f0_p0']

data_acc['focal_move_similarity_t4']= data_acc['Cos_f4_p2'] - data_acc['Cos_f2_p2']

data_acc['move_away_t2'] = np.where(data_acc['peer_move_similarity_t2'] < 0, 1, 0)

data_acc['move_toward_t2'] = np.where(data_acc['peer_move_similarity_t2'] > 0, 1, 0)


In [245]:
# Robustness

data_acc['focal_move_similarity_t3']= data_acc['Cos_f3_p1'] - data_acc['Cos_f1_p1']

data_acc['peer_move_similarity_t2']= data_acc['Cos_f0_p2'] - data_acc['Cos_f0_p0']

data_acc['focal_move_similarity_t4']= data_acc['Cos_f4_p2'] - data_acc['Cos_f2_p2']

data_acc['move_away_t2'] = np.where(data_acc['peer_move_similarity_t2'] < 0, 1, 0)

data_acc['move_toward_t2'] = np.where(data_acc['peer_move_similarity_t2'] > 0, 1, 0)


# Pair codes for the fixed pair and group year effect

In [246]:
# Sort GVKEYs within each dyad
data_acc['min_GVKEY'] = data_acc[['GVKEY focal', 'GVKEY peer']].min(axis=1)
data_acc['max_GVKEY'] = data_acc[['GVKEY focal', 'GVKEY peer']].max(axis=1)

# Create a unique dyad identifier
data_acc ['dyad_GVKEY'] = data_acc ['min_GVKEY'].astype(str) + "_" + data_acc['max_GVKEY'].astype(str)

---

# Heterogeneity & R&D intensity at the level of naicsh

## Heterogeneity naicsh6:
### Calculate the standard deviation of each embedding in a given year and naicsh6, then sum all the standard deviations

## Exclude firms firms with no analyst coverage

In [248]:
wrds_vector_filter.iloc[0:1]

,GVKEY,datadate,cik,conm,rdate,fdate,file_id,regsic,cusip_8,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,embedding_10,embedding_11,embedding_12,embedding_13,embedding_14,embedding_15,embedding_16,embedding_17,embedding_18,embedding_19,embedding_20,embedding_21,embedding_22,embedding_23,embedding_24,embedding_25,embedding_26,embedding_27,embedding_28,embedding_29,embedding_30,embedding_31,embedding_32,embedding_33,embedding_34,embedding_35,embedding_36,embedding_37,embedding_38,embedding_39,embedding_40,embedding_41,embedding_42,embedding_43,embedding_44,embedding_45,embedding_46,embedding_47,embedding_48,embedding_49,embedding_50,embedding_51,embedding_52,embedding_53,embedding_54,embedding_55,embedding_56,embedding_57,embedding_58,embedding_59,embedding_60,embedding_61,embedding_62,embedding_63,embedding_64,embedding_65,embedding_66,embedding_67,embedding_68,embedding_69,embedding_70,embedding_71,embedding_72,embedding_73,embedding_74,embedding_75,embedding_76,embedding_77,embedding_78,embedding_79,embedding_80,embedding_81,embedding_82,embedding_83,embedding_84,embedding_85,embedding_86,embedding_87,embedding_88,embedding_89,embedding_90,embedding_91,embedding_92,embedding_93,embedding_94,embedding_95,embedding_96,embedding_97,embedding_98,embedding_99,embedding_100,embedding_101,embedding_102,embedding_103,embedding_104,embedding_105,embedding_106,embedding_107,embedding_108,embedding_109,embedding_110,embedding_111,embedding_112,embedding_113,embedding_114,embedding_115,embedding_116,embedding_117,embedding_118,embedding_119,embedding_120,embedding_121,embedding_122,embedding_123,embedding_124,embedding_125,embedding_126,embedding_127,embedding_128,embedding_129,embedding_130,embedding_131,embedding_132,embedding_133,embedding_134,embedding_135,embedding_136,embedding_137,embedding_138,embedding_139,embedding_140,embedding_141,embedding_142,embedding_143,embedding_144,embedding_145,embedding_146,embedding_147,embedding_148,embedding_149,embedding_150,embedding_151,embedding_152,embedding_153,embedding_154,embedding_155,embedding_156,embedding_157,embedding_158,embedding_159,embedding_160,embedding_161,embedding_162,embedding_163,embedding_164,embedding_165,embedding_166,embedding_167,embedding_168,embedding_169,embedding_170,embedding_171,embedding_172,embedding_173,embedding_174,embedding_175,embedding_176,embedding_177,embedding_178,embedding_179,embedding_180,embedding_181,embedding_182,embedding_183,embedding_184,embedding_185,embedding_186,embedding_187,embedding_188,embedding_189,embedding_190,embedding_191,embedding_192,embedding_193,embedding_194,embedding_195,embedding_196,embedding_197,embedding_198,embedding_199,embedding_200,embedding_201,embedding_202,embedding_203,embedding_204,embedding_205,embedding_206,embedding_207,embedding_208,embedding_209,embedding_210,embedding_211,embedding_212,embedding_213,embedding_214,embedding_215,embedding_216,embedding_217,embedding_218,embedding_219,embedding_220,embedding_221,embedding_222,embedding_223,embedding_224,embedding_225,embedding_226,embedding_227,embedding_228,embedding_229,embedding_230,embedding_231,embedding_232,embedding_233,embedding_234,embedding_235,embedding_236,embedding_237,embedding_238,embedding_239,embedding_240,embedding_241,embedding_242,embedding_243,embedding_244,embedding_245,embedding_246,embedding_247,embedding_248,embedding_249,embedding_250,embedding_251,embedding_252,embedding_253,embedding_254,embedding_255,embedding_256,embedding_257,embedding_258,embedding_259,embedding_260,embedding_261,embedding_262,embedding_263,embedding_264,embedding_265,embedding_266,embedding_267,embedding_268,embedding_269,embedding_270,embedding_271,embedding_272,embedding_273,embedding_274,embedding_275,embedding_276,embedding_277,embedding_278,embedding_279,embedding_280,embedding_281,embedding_282,embedding_283,embedding_284,embedding_285,embedding_286,embedding_287,embedding_288,embe

## Remove firms with no analyst coverage

In [249]:
wrds_vector_filter.rename(columns={'cusip_8': 'CUSIP'}, inplace=True)
wrds_vector_filter = pd.merge(wrds_vector_filter, analysts_year_firm_filtered, on=['CUSIP', 'datadate'], how='inner')

# Calculation:

In [250]:
embedding_cols = [column for column in wrds_vector_filter.columns if 'embedding' in column]

std_devs6 = wrds_vector_filter.groupby(['year_modified', 'naicsh6'])[embedding_cols].std()

hetero_naicsh6 = std_devs6.sum(axis=1).reset_index(name='hetero_naicsh6')

# Focal industry heterogeneity

data_acc= pd.merge(data_acc, hetero_naicsh6, right_on=['year_modified', 'naicsh6'],
                   left_on= ['year_modified', 'naicsh6 focal'], how= 'left')

data_acc.drop('naicsh6',axis=1, inplace=True)

data_acc = data_acc.rename(columns={'hetero_naicsh6': 'hetero_naicsh6_focal'})

# Peer industry heterogeneity

data_acc= pd.merge(data_acc, hetero_naicsh6, right_on=['year_modified', 'naicsh6'],
                   left_on= ['year_modified', 'naicsh6 peer'], how= 'left')

data_acc.drop('naicsh6',axis=1, inplace=True)

data_acc = data_acc.rename(columns={'hetero_naicsh6': 'hetero_naicsh6_peer'})


## Heterogeneity naicsh 4

In [251]:
std_devs4 = wrds_vector_filter.groupby(['year_modified', 'naicsh4'])[embedding_cols].std()

hetero_naicsh4 = std_devs4.sum(axis=1).reset_index(name='hetero_naicsh4')

# Focal industry heterogeneity

data_acc= pd.merge(data_acc, hetero_naicsh4, right_on=['year_modified', 'naicsh4'],
                   left_on= ['year_modified', 'naicsh4 focal'], how= 'left')

data_acc.drop('naicsh4',axis=1, inplace=True)

data_acc = data_acc.rename(columns={'hetero_naicsh4': 'hetero_naicsh4_focal'})

# Peer industry heterogeneity

data_acc= pd.merge(data_acc, hetero_naicsh4, right_on=['year_modified', 'naicsh4'],
                   left_on= ['year_modified', 'naicsh4 peer'], how= 'left')

data_acc.drop('naicsh4',axis=1, inplace=True)

data_acc = data_acc.rename(columns={'hetero_naicsh4': 'hetero_naicsh4_peer'})


## R&D Intensity naicsh4 and 6:

In [252]:
industry_intensity_4 = compustat_nocoverage_removed.groupby(['year_modified', 'naicsh4']).agg({
    'rd_intensity_win': 'mean'}).reset_index()

industry_intensity_6 = compustat_nocoverage_removed.groupby(['year_modified', 'naicsh6']).agg({
    'rd_intensity_win': 'mean'}).reset_index()

industry_intensity_4['naicsh4 focal']= industry_intensity_4['naicsh4']
industry_intensity_4['naicsh4 peer']= industry_intensity_4['naicsh4']
industry_intensity_4['rd_intensity_industry_4']= industry_intensity_4['rd_intensity_win']


industry_intensity_6['naicsh6 focal']= industry_intensity_6['naicsh6']
industry_intensity_6['naicsh6 peer']= industry_intensity_6['naicsh6']
industry_intensity_6['rd_intensity_industry_6']= industry_intensity_6['rd_intensity_win']


In [253]:
data_acc = pd.merge(data_acc, industry_intensity_4[['year_modified', 'rd_intensity_industry_4', 
                                                                         'naicsh4 focal']], on=['year_modified', 'naicsh4 focal'], how= 'left')

data_acc = pd.merge(data_acc, industry_intensity_6[['year_modified', 'rd_intensity_industry_6', 
                                                                         'naicsh6 focal']], on=['year_modified', 'naicsh6 focal'], how= 'left')

----

# Change names (change cosine to similarity)

In [255]:
# rename: cosine to similarity

data_acc.rename(columns={'Cos_f1_p0': 'similarity_f1_p0', 'Cos_f1_p1': 'similarity_f1_p1', 
                                     'Cos_f2_p1': 'similarity_f2_p1', 'Cos_f1_p2': 'similarity_f1_p2', 'Cos_f0_p0': 'similarity_f0_p0'
                                     , 'Cos_f0_p1':'similarity_f0_p1'	, 'Cos_f2_p2': 'similarity_f2_p2', 
                                     'Cos_f0_p2': 'similarity_f0_p2', 'Cos_f4_p2': 'similarity_f4_p2', 'Cos_f3_p1':'similarity_f3_p1' }	, inplace=True)

In [256]:
compustat_filtered['datadate focal']= pd.to_datetime(compustat_filtered['datadate focal'])
compustat_filtered['datadate peer']= pd.to_datetime(compustat_filtered['datadate peer'])

/var/folders/8d/hsy71hlj20q70n8qz1r3s4s00000gn/T/ipykernel_30107/282882535.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compustat_filtered['datadate focal']= pd.to_datetime(compustat_filtered['datadate focal'])
/var/folders/8d/hsy71hlj20q70n8qz1r3s4s00000gn/T/ipykernel_30107/282882535.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compustat_filtered['datadate peer']= pd.to_datetime(compustat_filtered['datadate peer'])


# HHI, Heterogeneity, and average for the most 25 and 5 similar firms

In [257]:
from datetime import date

today = date.today()

# Format the date as YYYYMMDD
formatted_date = today.strftime('%Y%m%d')

# Create a filename using the date
filename = f"data_25_{formatted_date}.csv"

data_25.to_csv(filename, index=False) 


In [258]:
# 25

data25 = pd.read_csv(filename)

data25['datadate focal'] = pd.to_datetime(data25['datadate focal'])

data25['datadate peer'] = pd.to_datetime(data25['datadate peer'])

data25= pd.merge(data25, compustat_filtered[['GVKEY focal', 'datadate focal' , 'sale', 'sale_win']], on=['GVKEY focal', 'datadate focal'] , how='left')

data25= pd.merge(data25, compustat_filtered[['GVKEY peer', 'datadate peer' , 'sale', 'sale_win']], on=['GVKEY peer', 'datadate peer'],how='left' )


# 5 
# We sort the dataset to include the five most similar ones:

df_sorted = data_acc.sort_values(by=['GVKEY focal', 'year_modified', 'similarity_f0_p0'], ascending=[True, True, False])

data5 = df_sorted.groupby(['GVKEY focal', 'year_modified']).head(5).reset_index(drop=True)

data5['datadate focal'] = pd.to_datetime(data5['datadate focal'])

data5['datadate peer'] = pd.to_datetime(data5['datadate peer'])

In [259]:
# The sum of market share for each peer firm (Indeed, for each focal firm, we want to calcualte the sum of market share of all peer firms)

sale25 = data25.groupby(['GVKEY focal', 'year_modified'])['sale_y'].sum().reset_index(name=f'total_sales')
data25 = data25.merge(sale25, on=['GVKEY focal', 'year_modified'])
data25['market_share25'] = data25['sale_y'] / data25['total_sales']
data25['squared_market_share25'] = data25['market_share25'] ** 2
hhi25 = data25.groupby(['GVKEY focal', 'year_modified'])['squared_market_share25'].sum().reset_index(name='hhi25')
hhi25['hhi25_inv']= 1- hhi25['hhi25']


In [260]:
data_acc = pd.merge(data_acc, hhi25, on=['GVKEY focal', 'year_modified'], how= 'left')

In [261]:
sale5 = data5.groupby(['GVKEY focal', 'year_modified'])['sale_y'].sum().reset_index(name=f'total_sales')
data5 = data5.merge(sale5, on=['GVKEY focal', 'year_modified'])
data5['market_share5'] = data5['sale_y'] / data5['total_sales']
data5['squared_market_share5'] = data5['market_share5'] ** 2
hhi5 = data5.groupby(['GVKEY focal', 'year_modified'])['squared_market_share5'].sum().reset_index(name='hhi5')
hhi5 ['hhi5_inv']= 1- hhi5['hhi5']

data_acc = pd.merge(data_acc, hhi5, on=['GVKEY focal', 'year_modified'], how= 'left')

# Moving toward the industry control:

In [262]:
wrds_vector_filter['GVKEY peer']= wrds_vector_filter['GVKEY']

wrds_vector_filter['datadate peer']= wrds_vector_filter['datadate']

wrds_vector_filter= wrds_vector_filter.drop_duplicates(subset=['GVKEY', 'year_modified'], keep='first')

## Average vector of 25 at t1 and t0:

In [263]:
# t1

data25_t1 = pd.read_csv(filename)

data25_t1['datadate peer'] = pd.to_datetime(data25_t1['datadate peer'])
data25_t1['datadate focal'] = pd.to_datetime(data25_t1['datadate focal'])

data25_t1 = data25_t1.merge(wrds_vector_filter, on=['GVKEY peer', 'datadate peer'])

data25_t1.rename(columns={'year_modified_x': 'year_modified'}, inplace=True)

data25_t1.drop(columns=['year_modified_y'], inplace=True)

embedding_columns = [f'embedding_{i}' for i in range(768)]

average_peers_t1 = data25_t1.groupby(['GVKEY focal', 'datadate focal', 'year_modified'])[embedding_columns].mean().reset_index()

In [264]:
# t0 
data25_t0= pd.read_csv(filename)

data25_t0['year_modified']= data25_t0['year_modified'] - 1

data25_t0 = data25_t0.merge(wrds_vector_filter, on=['GVKEY peer', 'year_modified'])

embedding_columns = [f'embedding_{i}' for i in range(768)]

average_peers_t0 = data25_t0.groupby(['GVKEY focal', 'datadate focal','year_modified'])[embedding_columns].mean().reset_index()

In [265]:
focal_industry_similarity = {'year_modified':[], 'GVKEY focal':[], 'legitimacy':[], 'moving_toward_industry':[]}

In [266]:
focal_year= data_acc.groupby(['GVKEY focal', 'year_modified']).size().reset_index()

peer_year= data_acc.groupby(['GVKEY peer', 'year_modified', 'GVKEY focal']).size().reset_index()

Focal year was created from the merged_rec which considers the year_modified + 1


In [267]:
from sklearn.metrics.pairwise import cosine_similarity

In [268]:

for year in range (1994, 2022):
    
    for focal in focal_year[focal_year['year_modified'] == year]['GVKEY focal'].unique():
        
        focal_industry_similarity['year_modified'].append(year)
        
        focal_industry_similarity['GVKEY focal'].append(focal)
        
        embeddings_focal_t2 = wrds_vector_filter[(wrds_vector_filter['year_modified'] == year+1) & (wrds_vector_filter['GVKEY'] == focal)][embedding_columns].values
        embeddings_focal_t1 = wrds_vector_filter[(wrds_vector_filter['year_modified'] == year) & (wrds_vector_filter['GVKEY'] == focal)][embedding_columns].values

        industry_t0 = average_peers_t0 [(average_peers_t0['year_modified'] == year-1) & (average_peers_t0['GVKEY focal'] == focal)][embedding_columns].values   
        industry_t1 = average_peers_t1 [(average_peers_t1['year_modified'] == year) & (average_peers_t1['GVKEY focal'] == focal)][embedding_columns].values   

        similarity_f1i1 = cosine_similarity(embeddings_focal_t1, industry_t1)[0][0]
        similarity_f2i1 = cosine_similarity(embeddings_focal_t2, industry_t1)[0][0]
        
        focal_industry_similarity ['legitimacy'].append(similarity_f1i1)
        
        moving_toward_industry = similarity_f2i1 - similarity_f1i1        
        focal_industry_similarity['moving_toward_industry'].append(moving_toward_industry)
        
        

In [269]:
df_focal_industry_similarity = pd.DataFrame(focal_industry_similarity)

df_focal_industry_similarity.iloc[0:1]

,year_modified,GVKEY focal,legitimacy,moving_toward_industry
0,1994,1240,0.824449,0.011689


# Homogeneity 25 firms: The average of similarities between a focal firm and its peers

In [270]:
hetero25 = data25.groupby(['GVKEY focal', 'year_modified'])['Cos_f1_p1'].mean().reset_index()

hetero25.rename(columns={'Cos_f1_p1': 'hetero25'}, inplace=True)

In [271]:
data_acc= pd.merge(data_acc, hetero25, on=['GVKEY focal', 'year_modified'], how= 'left')

data_acc = pd.merge(data_acc, df_focal_industry_similarity, on=['GVKEY focal', 'year_modified'], how= 'left')


---

# R&D intensity of the 25, 10, and 5 most similar firms around a focal firm

## R&D intensity 25:

In [273]:
data25_rd= pd.merge(data_25, compustat_filtered[['GVKEY peer', 'datadate peer' , 'rd_intensity_win', 'rd_intensity']], on=['GVKEY peer', 'datadate peer'], how= 'left')

rd25 = data25_rd.groupby(['GVKEY focal', 'year_modified'])[['rd_intensity_win','rd_intensity']].mean().reset_index()

rd25.rename(columns={'rd_intensity_win': 'rd_25_win', 'rd_intensity':'rd_25'}, inplace=True)

data_acc = pd.merge(data_acc, rd25, on=['GVKEY focal', 'year_modified'], how= 'left')

## R&D 15:

In [274]:
data_25_rd15 = data_25.groupby(['GVKEY focal', 'year_modified']).head(15)

data_25_rd15= pd.merge(data_25_rd15, compustat_filtered[['GVKEY peer', 'datadate peer' , 'rd_intensity_win', 'rd_intensity']], on=['GVKEY peer', 'datadate peer'], how= 'left')

rd15 = data_25_rd15.groupby(['GVKEY focal', 'year_modified'])[['rd_intensity_win','rd_intensity']].mean().reset_index()

rd15.rename(columns={'rd_intensity_win': 'rd_15_win', 'rd_intensity':'rd_15'}, inplace=True)

data_acc = pd.merge(data_acc, rd15, on=['GVKEY focal', 'year_modified'], how= 'left')

# R&D 5 & 10

In [275]:
data5.iloc[0:2]

,year_modified,datadate focal,fdate focal,GVKEY focal,GVKEY peer,similarity_f1_p0,similarity_f1_p1,similarity_f0_p0,similarity_f0_p1,similarity_f2_p1,similarity_f2_p2,similarity_f1_p2,similarity_f0_p2,similarity_f4_p2,similarity_f3_p1,naicsh6 focal,naicsh4 focal,naicsh2 focal,CUSIP focal,naicsh6 peer,naicsh4 peer,naicsh2 peer,CUSIP peer,fdate peer,datadate peer,after_filing_90_peer,before_filing_90_peer,after_filing_30_peer,before_filing_30_peer,after_filing_60_peer,before_filing_60_peer,after_previous_filing_90_peer,after_previous_filing_60_peer,after_previous_filing_30_peer,before_filing_360_peer,after_previous_2year_30_peer,previous_2year_peer,nextyear_30_peer,nextyear_60_peer,after_filing_90_w_peer,before_filing_90_w_peer,after_filing_30_w_peer,before_filing_30_w_peer,after_filing_60_w_peer,before_filing_60_w_peer,after_previous_filing_90_w_peer,after_previous_filing_60_w_peer,after_previous_filing_30_w_peer,before_filing_360_w_peer,after_previous_2year_30_w_peer,previous_2year_w_peer,nextyear_30_w_peer,nextyear_60_w_peer,up_after_90,down_after_90,up_after_60,down_after_60,up_after_30,down_after_30,up_after_180,down_after_180,up_after_360,down_after_360,meanrec_diff_90_w,meanrec_diff_30_w,meanrec_diff_60_w,meanrec_diff_90_360_w,meanrec_diff_30_360_w,meanrec_diff_90_previous_w,meanrec_diff_60_previous_w,meanrec_diff_30_previous_w,meanrec_diff_next_30_w,meanrec_diff_next_60_w,meanrec_diff_before_previous_30_w,meanrec_diff_before_previous_360_w,meanrec_diff_90,meanrec_diff_30,meanrec_diff_60,meanrec_diff_90_360,meanrec_diff_30_360,meanrec_diff_90_previous,meanrec_diff_60_previous,meanrec_diff_30_previous,meanrec_diff_next_30,meanrec_diff_next_60,meanrec_diff_before_previous_30,meanrec_diff_before_previous_360,after_filing_90_focal,before_filing_90_focal,after_filing_30_focal,before_filing_30_focal,after_filing_60_focal,before_filing_60_focal,after_previous_filing_90_focal,after_previous_filing_60_focal,after_previous_filing_30_focal,before_filing_360_focal,after_previous_2year_30_focal,previous_2year_focal,nextyear_30_focal,nextyear_60_focal,after_filing_90_w_focal,before_filing_90_w_focal,after_filing_30_w_focal,before_filing_30_w_focal,after_filing_60_w_focal,before_filing_60_w_focal,after_previous_filing_90_w_focal,after_previous_filing_60_w_focal,after_previous_filing_30_w_focal,before_filing_360_w_focal,after_previous_2year_30_w_focal,previous_2year_w_focal,nextyear_30_w_focal,nextyear_60_w_focal,meanrec_diff_90_focal,meanrec_diff_30_focal,meanrec_diff_60_focal,meanrec_diff_90_360_focal,meanrec_diff_30_360_focal,meanrec_diff_90_previous_focal,meanrec_diff_60_previous_focal,meanrec_diff_30_previous_focal,meanrec_diff_next_30_focal,meanrec_diff_next_60_focal,meanrec_diff_before_previous_30_focal,meanrec_diff_before_previous_360_focal,meanrec_diff_90_w_focal,meanrec_diff_30_w_focal,meanrec_diff_60_w_focal,meanrec_diff_90_360_w_focal,meanrec_diff_30_360_w_focal,meanrec_diff_90_previous_w_focal,meanrec_diff_60_previous_w_focal,meanrec_diff_30_previous_w_focal,meanrec_diff_next_30_w_focal,meanrec_diff_next_60_w_focal,meanrec_diff_before_previous_30_w_focal,meanrec_diff_before_previous_360_w_focal,car1,car3,car5,car7,n_emp_x,log_n_emp_x,hhi_naicsh6_inv_x,hhi_naicsh4_inv_x,hhi_naicsh6_inv_win_x,hhi_naicsh4_inv_win_x,roa_x,roa_win_x,roe_x,roe_win_x,sales_growth_naicsh4_x,sales_growth_naicsh4_win_x,rd_intensity_x,rd_intensity_win_x,naicsh4_x,sale_x,sale_win_x,slack_avail_x,slack_avail_win_x,adv_f_x,adv_f_win_x,intang_f_x,intang_f_win_x,dpt_f_x,dpt_f_win_x,leverage_x,leverage_win_x,tobins_q_x,tobins_q_win_x,market_share_naicsh4_x,prcc_f_x,EPS_x,EPS_win_x,log_at_win_x,at_x,log_at_x,ni_x,rd_f_x,mergers_x,mergers_win_x,market_share_naicsh6_x,market_share_naicsh4_win_x,naicsh6_x,ros_win_x,ros_x,n_emp_y,log_n_emp_y,hhi_naicsh6_inv_y,hhi_naicsh4_inv_y,hhi_naicsh6_inv_win_y,hhi_naicsh4_inv_win_y,roa_y,roa_win_y,roe_y,roe_win_y,sales_growth_naicsh4_y,sales_growth_naicsh4_win_y,rd_intensity_y,rd_intensity_win_y,naicsh4_y,sale_y,sale_win_y,s

In [276]:
# R&D 5

rd5 = data5.groupby(['GVKEY focal', 'year_modified'])[['rd_intensity_win_y','rd_intensity_y']].mean().reset_index()

rd5.rename (columns={'rd_intensity_win_y': 'rd_5_win',
                     'rd_intensity_y': 'rd_5'}, inplace=True)

data_acc = pd.merge(data_acc, rd5, on=['GVKEY focal', 'year_modified'], how= 'left')


# R&D 10

rd10 = data_acc.groupby(['GVKEY focal', 'year_modified'])[['rd_intensity_win_y','rd_intensity_y']].mean().reset_index()

rd10.rename (columns={'rd_intensity_win_y': 'rd_10_win', 
                     'rd_intensity_y': 'rd_10'}, inplace=True)

data_acc = pd.merge(data_acc, rd10, on=['GVKEY focal', 'year_modified'], how= 'left')

---

# Focal firm typicality for naicsh4 and 5

In [278]:
wrds_vector_filter['GVKEY focal']= wrds_vector_filter['GVKEY']

# Define a dectionary to add all firms' typicality
typ_all = {'year_modified':[], 'GVKEY':[], 'typicality_4':[] , 'typicality_5':[]}

wrds_vector_filter['naicsh5'] = wrds_vector_filter['naicsh6'].astype(str).str[:5]

In [279]:
# Average vectors for naicsh4 and 5 at t1:
average_naicsh4_t1 = wrds_vector_filter.groupby(['naicsh4', 'year_modified'])[embedding_cols].mean().reset_index()
average_naicsh5_t1= wrds_vector_filter.groupby(['naicsh5', 'year_modified'])[embedding_cols].mean().reset_index()

In [280]:
for year in wrds_vector_filter['year_modified'].unique():
    
    for gvkey in wrds_vector_filter[wrds_vector_filter['year_modified'] == year]['GVKEY'].unique():
        
        naicsh4_focal = wrds_vector_filter[(wrds_vector_filter['year_modified'] == year) & (wrds_vector_filter['GVKEY'] == gvkey)]['naicsh4'].values[0]
        naicsh5_focal = wrds_vector_filter[(wrds_vector_filter['year_modified'] == year) & (wrds_vector_filter['GVKEY'] == gvkey)]['naicsh5'].values[0]
        
        focal_vector_t1 = wrds_vector_filter[(wrds_vector_filter['year_modified'] == year) & (wrds_vector_filter['GVKEY'] == gvkey)][embedding_cols].values
        
       # focal_vector_t2 = wrds_vector_filter[(wrds_vector_filter['year_modified'] == year+1) & (wrds_vector_filter['GVKEY'] == gvkey)][embedding_cols].values

        average_naich5 = average_naicsh5_t1[(average_naicsh5_t1['year_modified'] == year) & (average_naicsh5_t1['naicsh5'] == naicsh5_focal)][embedding_cols].values
        
        average_naicsh4 = average_naicsh4_t1[(average_naicsh4_t1['year_modified'] == year) & (average_naicsh4_t1['naicsh4'] == naicsh4_focal)][embedding_cols].values
        
        typicality4 = cosine_similarity(focal_vector_t1, average_naicsh4)[0][0]
        typicality5 = cosine_similarity(focal_vector_t1, average_naich5)[0][0]
        
        typ_all['year_modified'].append(year)
        typ_all['GVKEY'].append(gvkey)
        typ_all['typicality_4'].append(typicality4)
        typ_all['typicality_5'].append(typicality5)
   
        

In [281]:
typ_all = pd.DataFrame(typ_all)
typ_all.iloc[0:5]

,year_modified,GVKEY,typicality_4,typicality_5
0,1993,1004,0.956828,1.000000
1,1993,1055,0.946706,0.952710
2,1993,1078,0.944919,0.944919
3,1993,1194,0.926777,0.935263
4,1993,1203,1.000000,1.000000


In [282]:
typ_all['year_modified_t2']= typ_all['year_modified']
typ_all['GVKEY peer']= typ_all['GVKEY']
typ_all ['GVKEY focal']= typ_all['GVKEY']
data_acc['year_modified_t2']= data_acc['year_modified'] +1 

In [283]:
# Focal t1
data_acc= pd.merge(data_acc, typ_all [['GVKEY focal','typicality_4', 'typicality_5', 'year_modified']], on=['year_modified', 'GVKEY focal'], how= 'left')

data_acc.rename(
    columns={
        'typicality_4': 'typicality_4_focal_t1',
        'typicality_5': 'typicality_5_focal_t1'},inplace=True)

# Focal t2
data_acc= pd.merge(data_acc, typ_all [['GVKEY focal','typicality_4', 'typicality_5', 'year_modified_t2']], on=['year_modified_t2', 'GVKEY focal'], how= 'left')
                                                                                                                                                    
data_acc.rename(
    columns={
        'typicality_4': 'typicality_4_focal_t2',
        'typicality_5': 'typicality_5_focal_t2'},inplace=True)

# Peer t1
data_acc= pd.merge(data_acc, typ_all [['GVKEY peer','typicality_4', 'typicality_5', 'year_modified']], on=['year_modified', 'GVKEY peer'], how= 'left')

data_acc.rename(
    columns={
        'typicality_4': 'typicality_4_peer_t1',
        'typicality_5': 'typicality_5_peer_t1'},inplace=True)

# Peer t2
data_acc= pd.merge(data_acc, typ_all [['GVKEY peer','typicality_4', 'typicality_5', 'year_modified_t2']], on=['year_modified_t2', 'GVKEY peer'], how= 'left')

data_acc.rename(
    columns={
        'typicality_4': 'typicality_4_peer_t2',
        'typicality_5': 'typicality_5_peer_t2'},inplace=True)


In [284]:
data_acc.iloc[0:2]

,year_modified,datadate focal,fdate focal,GVKEY focal,GVKEY peer,similarity_f1_p0,similarity_f1_p1,similarity_f0_p0,similarity_f0_p1,similarity_f2_p1,similarity_f2_p2,similarity_f1_p2,similarity_f0_p2,similarity_f4_p2,similarity_f3_p1,naicsh6 focal,naicsh4 focal,naicsh2 focal,CUSIP focal,naicsh6 peer,naicsh4 peer,naicsh2 peer,CUSIP peer,fdate peer,datadate peer,after_filing_90_peer,before_filing_90_peer,after_filing_30_peer,before_filing_30_peer,after_filing_60_peer,before_filing_60_peer,after_previous_filing_90_peer,after_previous_filing_60_peer,after_previous_filing_30_peer,before_filing_360_peer,after_previous_2year_30_peer,previous_2year_peer,nextyear_30_peer,nextyear_60_peer,after_filing_90_w_peer,before_filing_90_w_peer,after_filing_30_w_peer,before_filing_30_w_peer,after_filing_60_w_peer,before_filing_60_w_peer,after_previous_filing_90_w_peer,after_previous_filing_60_w_peer,after_previous_filing_30_w_peer,before_filing_360_w_peer,after_previous_2year_30_w_peer,previous_2year_w_peer,nextyear_30_w_peer,nextyear_60_w_peer,up_after_90,down_after_90,up_after_60,down_after_60,up_after_30,down_after_30,up_after_180,down_after_180,up_after_360,down_after_360,meanrec_diff_90_w,meanrec_diff_30_w,meanrec_diff_60_w,meanrec_diff_90_360_w,meanrec_diff_30_360_w,meanrec_diff_90_previous_w,meanrec_diff_60_previous_w,meanrec_diff_30_previous_w,meanrec_diff_next_30_w,meanrec_diff_next_60_w,meanrec_diff_before_previous_30_w,meanrec_diff_before_previous_360_w,meanrec_diff_90,meanrec_diff_30,meanrec_diff_60,meanrec_diff_90_360,meanrec_diff_30_360,meanrec_diff_90_previous,meanrec_diff_60_previous,meanrec_diff_30_previous,meanrec_diff_next_30,meanrec_diff_next_60,meanrec_diff_before_previous_30,meanrec_diff_before_previous_360,after_filing_90_focal,before_filing_90_focal,after_filing_30_focal,before_filing_30_focal,after_filing_60_focal,before_filing_60_focal,after_previous_filing_90_focal,after_previous_filing_60_focal,after_previous_filing_30_focal,before_filing_360_focal,after_previous_2year_30_focal,previous_2year_focal,nextyear_30_focal,nextyear_60_focal,after_filing_90_w_focal,before_filing_90_w_focal,after_filing_30_w_focal,before_filing_30_w_focal,after_filing_60_w_focal,before_filing_60_w_focal,after_previous_filing_90_w_focal,after_previous_filing_60_w_focal,after_previous_filing_30_w_focal,before_filing_360_w_focal,after_previous_2year_30_w_focal,previous_2year_w_focal,nextyear_30_w_focal,nextyear_60_w_focal,meanrec_diff_90_focal,meanrec_diff_30_focal,meanrec_diff_60_focal,meanrec_diff_90_360_focal,meanrec_diff_30_360_focal,meanrec_diff_90_previous_focal,meanrec_diff_60_previous_focal,meanrec_diff_30_previous_focal,meanrec_diff_next_30_focal,meanrec_diff_next_60_focal,meanrec_diff_before_previous_30_focal,meanrec_diff_before_previous_360_focal,meanrec_diff_90_w_focal,meanrec_diff_30_w_focal,meanrec_diff_60_w_focal,meanrec_diff_90_360_w_focal,meanrec_diff_30_360_w_focal,meanrec_diff_90_previous_w_focal,meanrec_diff_60_previous_w_focal,meanrec_diff_30_previous_w_focal,meanrec_diff_next_30_w_focal,meanrec_diff_next_60_w_focal,meanrec_diff_before_previous_30_w_focal,meanrec_diff_before_previous_360_w_focal,car1,car3,car5,car7,n_emp_x,log_n_emp_x,hhi_naicsh6_inv_x,hhi_naicsh4_inv_x,hhi_naicsh6_inv_win_x,hhi_naicsh4_inv_win_x,roa_x,roa_win_x,roe_x,roe_win_x,sales_growth_naicsh4_x,sales_growth_naicsh4_win_x,rd_intensity_x,rd_intensity_win_x,naicsh4_x,sale_x,sale_win_x,slack_avail_x,slack_avail_win_x,adv_f_x,adv_f_win_x,intang_f_x,intang_f_win_x,dpt_f_x,dpt_f_win_x,leverage_x,leverage_win_x,tobins_q_x,tobins_q_win_x,market_share_naicsh4_x,prcc_f_x,EPS_x,EPS_win_x,log_at_win_x,at_x,log_at_x,ni_x,rd_f_x,mergers_x,mergers_win_x,market_share_naicsh6_x,market_share_naicsh4_win_x,naicsh6_x,ros_win_x,ros_x,n_emp_y,log_n_emp_y,hhi_naicsh6_inv_y,hhi_naicsh4_inv_y,hhi_naicsh6_inv_win_y,hhi_naicsh4_inv_win_y,roa_y,roa_win_y,roe_y,roe_win_y,sales_growth_naicsh4_y,sales_growth_naicsh4_win_y,rd_intensity_y,rd_intensity_win_y,naicsh4_y,sale_y,sale_win_y,s

----

# Strategy unqiueness Litov

In [286]:
ind_col = 'naicsh4' 
ind_type = 'naicsh' # 'naicsh' or 'sich'

In [ ]:
if ind_type == 'naicsh':
    seg_ind = 'NAICSS1'
elif ind_type == 'sich':
    seg_ind = 'SICS1'

segments = pd.read_csv('/.../compustat_segments2.csv')
# segments['year'] = segments['datadate'].str.slice(0,4).astype(int)

segments['datadate']= pd.to_datetime(segments['datadate'])

segments['yeard']= segments['datadate'].dt.year

segments['monthd']= segments['datadate'].dt.month

segments ['year'] = segments.apply(modify_year, axis=1)

## Remove firms with no analyst coverage from the segment file:

In [298]:
segments['datadate'] = pd.to_datetime(segments['datadate'])
analysts_year_firm_filtered['datadate'] = pd.to_datetime(analysts_year_firm_filtered['datadate'])
segments.rename(columns={'cusip': 'CUSIP'}, inplace=True)
segments['CUSIP'] = segments['CUSIP'].apply(lambda x: x[:-1])

segments = pd.merge(segments, analysts_year_firm_filtered, on=['CUSIP', 'datadate'], how='inner')

In [299]:
segments = segments[(segments['stype'] == 'BUSSEG') | (segments['stype'] == 'OPSEG')]
segments = segments[['gvkey', 'year', 'sales', seg_ind ]]
segments = segments[segments['sales'] > 0]
segments = segments[segments[seg_ind].notnull()]
segments = segments[segments[seg_ind] != '']
segments = segments[segments[seg_ind] != 0]
segments[seg_ind] = segments[seg_ind].astype(int)
segments['year'] = segments['year'].astype(int)
segments['gvkey'] = segments['gvkey'].astype(int)
segments = segments.rename(columns={'gvkey': 'GVKEY'})
segments = segments.rename(columns={seg_ind: 'segment_sic'})
segments = segments.rename(columns={'sales': 'segment_sale'})
segments = segments.groupby(['GVKEY', 'year', 'segment_sic'])['segment_sale'].sum().reset_index(name='segment_sale')
segments['segment_sic'] = segments['segment_sic'].astype(int)

In [300]:
# Step 1
idx = segments.groupby(['GVKEY', 'year'])['segment_sale'].idxmax()
segments['primary_sic'] = segments.loc[idx, 'segment_sic']
segments['primary_sic'] = segments.groupby(['GVKEY', 'year'])['primary_sic'].transform('max')

# Step 2
total_sales = segments.groupby(['GVKEY', 'year'])['segment_sale'].transform('sum')
segments['norm_sale'] = segments['segment_sale'] / total_sales

# Step 3
firm_year_matrix = segments.pivot_table(index=['GVKEY', 'year', 'primary_sic'],
                                        columns='segment_sic',
                                        values='norm_sale').fillna(0)

# Step 4
actual_sales_matrix = segments.pivot_table(index=['GVKEY', 'year', 'primary_sic'],
                                           columns='segment_sic',
                                           values='segment_sale').fillna(0)

industry_year_sales = actual_sales_matrix.groupby(['primary_sic', 'year']).sum()

# Step 5
total_industry_sales = industry_year_sales.sum(axis=1)
norm_industry_year_sales = industry_year_sales.div(total_industry_sales, axis=0)

# Step 6
diff_matrix = firm_year_matrix.subtract(norm_industry_year_sales, axis=1)

# step 7: sum of squared differences
squared_diff_matrix = diff_matrix ** 2
sum_squared_diff = squared_diff_matrix.sum(axis=1)

uniqueness = sum_squared_diff.reset_index(name='strategy_unique')

In [301]:
uniqueness.iloc[0:2]

,GVKEY,year,primary_sic,strategy_unique
0,17225,1993,311.0,0.036567
1,23798,1993,311.0,0.353931


In [302]:
# Focal uniqueness
 
data_acc= pd.merge(data_acc, uniqueness, left_on=['GVKEY focal', 'year_modified'], 
                   right_on= ['GVKEY', 'year'], how='left')

data_acc.drop(['GVKEY', 'year'],axis=1, inplace=True)

data_acc = data_acc.rename(columns={'strategy_unique': 'strategy_unique_focal',
                                    'primary_sic': 'primary_sic_focal'})

# Peer uniqueness

data_acc= pd.merge(data_acc, uniqueness, left_on=['GVKEY peer', 'year_modified'], 
                   right_on= ['GVKEY', 'year'], how='left')

data_acc.drop(['GVKEY', 'year'],axis=1, inplace=True)

data_acc = data_acc.rename(columns={'strategy_unique': 'strategy_unique_peer',
                                    'primary_sic': 'primary_sic_peer'})


---

# Change variables: Change size and average of changes:

## Change size:

In [310]:
wrds_vector_change = wrds_vector_filter.copy()

# List of embedding column names
embed_cols = ['embedding_' + str(i) for i in range(768)]

# Year diff
wrds_vector_change['year_diff'] = wrds_vector_change.groupby('GVKEY')['year_modified'].diff()

# Compute the difference in embeddings for each company
wrds_vector_change[embed_cols] = wrds_vector_change.groupby('GVKEY')[embed_cols].diff()

change = wrds_vector_change[wrds_vector_change['year_diff'] == 1]

change['change_size'] = change.apply(lambda row: np.linalg.norm(row[embed_cols]), axis=1)

change2= change

## 1 SD above the mean

In [315]:
# naicsh2
# Group by 'naicsh2' and 'year_modified', then compute mean and std for 'change_size'
industry_stats2 = change2.groupby(['naicsh2', 'year_modified'])['change_size'].agg(['mean', 'std']).reset_index()

# Rename columns for clarity
industry_stats2.rename(columns={'mean': 'industry_mean_change_naicsh2', 'std': 'industry_std_change_naicsh2'}, inplace=True)

# Join stats back to original df
change2 = pd.merge(change2, industry_stats2, on=['naicsh2', 'year_modified'], how='left')

# Create a new column for mean plus sd
change2['mean_plus_sd_naicsh2'] = change2 ['industry_mean_change_naicsh2'] + change2 ['industry_std_change_naicsh2']
# Keep rows where 'change_size' is greater than 'mean_plus_sd'

change2 ['change_naicsh2'] = np.where((change2['change_size'] > change2['mean_plus_sd_naicsh2']), 1, 0)


# naicsh4 
# Group by 'naicsh4' and 'year_modified', then compute mean and std for 'change_size'
industry_stats4 = change2.groupby(['naicsh4', 'year_modified'])['change_size'].agg(['mean', 'std']).reset_index()

# Rename columns for clarity
industry_stats4.rename(columns={'mean': 'industry_mean_change_naicsh4', 'std': 'industry_std_change_naicsh4'}, inplace=True)

# Join stats back to original df
change2 = pd.merge(change2, industry_stats4, on=['naicsh4', 'year_modified'], how='left')

# Create a new column for mean plus sd
change2['mean_plus_sd_naicsh4'] = change2 ['industry_mean_change_naicsh4'] + change2 ['industry_std_change_naicsh4']
# Keep rows where 'change_size' is greater than 'mean_plus_sd'

change2 ['change_naicsh4'] = np.where((change2['change_size'] > change2['mean_plus_sd_naicsh4']), 1, 0)


## 1 SD above the median:

In [316]:
industry_stats2_median = change2.groupby(['naicsh2', 'year_modified'])['change_size'].agg(['median', 'std']).reset_index()

industry_stats2_median.rename(columns={'median': 'industry_median_change_naicsh2', 'std': 'industry_std2_change_naicsh2'}, inplace=True)

change2 = pd.merge(change2, industry_stats2_median, on=['naicsh2', 'year_modified'], how='left')

change2['median_plus_sd_naicsh2'] = change2['industry_median_change_naicsh2'] + change2['industry_std2_change_naicsh2']

change2['change_naicsh2_median'] = np.where(change2['change_size'] > change2['median_plus_sd_naicsh2'], 1, 0)


industry_stats4_median = change2.groupby(['naicsh4', 'year_modified'])['change_size'].agg(['median', 'std']).reset_index()

industry_stats4_median.rename(columns={'median': 'industry_median_change_naicsh4', 'std': 'industry_std2_change_naicsh4'}, inplace=True)

change2 = pd.merge(change2, industry_stats4_median, on=['naicsh4', 'year_modified'], how='left')

change2['median_plus_sd_naicsh4'] = change2['industry_median_change_naicsh4'] + change2['industry_std2_change_naicsh4']

change2['change_naicsh4_median'] = np.where(change2['change_size'] > change2['median_plus_sd_naicsh4'], 1, 0)


In [317]:
# Peer firm
data_acc = pd.merge(data_acc , change2[['GVKEY', 'year_modified','change_size','change_naicsh2','industry_mean_change_naicsh2','industry_std_change_naicsh2',
                                        'change_naicsh4','industry_mean_change_naicsh4','industry_std_change_naicsh4', 'change_naicsh4_median', 'change_naicsh2_median']],
                       left_on=['GVKEY peer', 'year_modified'],right_on= ['GVKEY', 'year_modified'], how='left')

data_acc.drop('GVKEY',axis=1, inplace=True)

data_acc = data_acc.rename(columns={'change_size': 'change_size_peer',
                                    'change_naicsh2': 'change_naicsh2_peer',
                                    'industry_mean_change_naicsh2': 'industry_mean_change_naicsh2_peer',
                                    'industry_std_change_naicsh2': 'industry_std_change_naicsh2_peer',
                                    'change_naicsh4': 'change_naicsh4_peer',
                                    'change_naicsh4_median': 'change_naicsh4_median_peer',
                                    'change_naicsh2_median': 'change_naicsh2_median_peer',
                                    'industry_mean_change_naicsh4': 'industry_mean_change_naicsh4_peer',
                                    'industry_std_change_naicsh4': 'industry_std_change_naicsh4_peer'})

# Focal firm
data_acc = pd.merge(data_acc , change2[['GVKEY', 'year_modified','change_size','change_naicsh2','industry_mean_change_naicsh2','industry_std_change_naicsh2',
                                        'change_naicsh4','industry_mean_change_naicsh4','industry_std_change_naicsh4']],
                       left_on=['GVKEY focal', 'year_modified'],right_on= ['GVKEY', 'year_modified'], how='left')

data_acc.drop('GVKEY',axis=1, inplace=True)

data_acc = data_acc.rename(columns={'change_size': 'change_size_focal',
                                    'change_naicsh2': 'change_naicsh2_focal',
                                    'industry_mean_change_naicsh2': 'industry_mean_change_naicsh2_focal',
                                    'industry_std_change_naicsh2': 'industry_std_change_naicsh2_focal',
                                    'change_naicsh4_median': 'change_naicsh4_median_focal',
                                    'change_naicsh2_median': 'change_naicsh2_median_focal',
                                    'change_naicsh4': 'change_naicsh4_focal',
                                    'industry_mean_change_naicsh4': 'industry_mean_change_naicsh4_focal',
                                    'industry_std_change_naicsh4': 'industry_std_change_naicsh4_focal'})


## Average of change in the past three years:
### t-2 to t-1 | t-1 to t0 | t0 to t1 for the focal firm (focal firm repositioning from t1 to t2):
### We average only across three consecutive years. If one year's data is unavailable, we calculate the average using the data from the existing years. If data from only one of the three consecutive years is available, that year's change size is considered as the average for the three years.


In [320]:
change ['datadate'] = pd.to_datetime(change['datadate'])

change_average_data = change [['GVKEY','datadate','fdate','year_modified','change_size']]

change_average_data.sort_values(by=['GVKEY', 'year_modified'], inplace=True)

change_average_data['consecutive'] = change_average_data.groupby('GVKEY')['year_modified'].diff().fillna(1).eq(1)

# Create a group identifier for consecutive segments
change_average_data['group'] = (~change_average_data['consecutive']).cumsum()

# Calculating rolling mean only for consecutive years
temp_result = (change_average_data.groupby(['GVKEY', 'group'])['change_size']
               .rolling(window=3, min_periods=1)
               .mean())

# The rolling operation keeps the grouped index, so we need to reset it to align with the original DataFrame
temp_result = temp_result.reset_index()
temp_result.rename(columns={'change_size': 'avg_change_size'}, inplace=True)

# Now, merge this result back to your original DataFrame
change_average_data = change_average_data.merge(temp_result[['GVKEY', 'group', 'level_2', 'avg_change_size']], 
                                                how='left', 
                                                left_index=True, 
                                                right_on='level_2')

change_average_data = change_average_data[['GVKEY_x', 'change_size','year_modified', 'fdate', 'avg_change_size']]

change_average_data.rename(columns={'GVKEY_x': 'GVKEY', 
                                   'avg_change_size': 'avg_change_size_3y' }, inplace=True)

/var/folders/8d/hsy71hlj20q70n8qz1r3s4s00000gn/T/ipykernel_30107/4248798102.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  change ['datadate'] = pd.to_datetime(change['datadate'])
/var/folders/8d/hsy71hlj20q70n8qz1r3s4s00000gn/T/ipykernel_30107/4248798102.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  change_average_data.sort_values(by=['GVKEY', 'year_modified'], inplace=True)
/var/folders/8d/hsy71hlj20q70n8qz1r3s4s00000gn/T/ipykernel_30107/4248798102.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

In [321]:
change_average_data.iloc[12000:12004]

,GVKEY,change_size,year_modified,fdate,avg_change_size_3y
12000,10198,0.049054,2021,2022-03-23,0.060652
12001,10200,0.062446,1996,1997-06-06,0.062446
12002,10200,0.127547,1997,1998-06-05,0.094996
12003,10200,0.080796,1998,1999-06-17,0.090263


In [322]:
data_acc= pd.merge(data_acc , change_average_data[['GVKEY', 'year_modified','avg_change_size_3y']],
                       left_on=['GVKEY focal', 'year_modified'],right_on= ['GVKEY', 'year_modified'], how='left')

data_acc.drop('GVKEY',axis=1, inplace=True)

data_acc.rename(columns={'avg_change_size_3y': 'avg_change_size_3y_focal' }, inplace=True)


In [323]:
print(data_acc.shape[0])
data_acc.iloc[0:1]

348520


,year_modified,datadate focal,fdate focal,GVKEY focal,GVKEY peer,similarity_f1_p0,similarity_f1_p1,similarity_f0_p0,similarity_f0_p1,similarity_f2_p1,similarity_f2_p2,similarity_f1_p2,similarity_f0_p2,similarity_f4_p2,similarity_f3_p1,naicsh6 focal,naicsh4 focal,naicsh2 focal,CUSIP focal,naicsh6 peer,naicsh4 peer,naicsh2 peer,CUSIP peer,fdate peer,datadate peer,after_filing_90_peer,before_filing_90_peer,after_filing_30_peer,before_filing_30_peer,after_filing_60_peer,before_filing_60_peer,after_previous_filing_90_peer,after_previous_filing_60_peer,after_previous_filing_30_peer,before_filing_360_peer,after_previous_2year_30_peer,previous_2year_peer,nextyear_30_peer,nextyear_60_peer,after_filing_90_w_peer,before_filing_90_w_peer,after_filing_30_w_peer,before_filing_30_w_peer,after_filing_60_w_peer,before_filing_60_w_peer,after_previous_filing_90_w_peer,after_previous_filing_60_w_peer,after_previous_filing_30_w_peer,before_filing_360_w_peer,after_previous_2year_30_w_peer,previous_2year_w_peer,nextyear_30_w_peer,nextyear_60_w_peer,up_after_90,down_after_90,up_after_60,down_after_60,up_after_30,down_after_30,up_after_180,down_after_180,up_after_360,down_after_360,meanrec_diff_90_w,meanrec_diff_30_w,meanrec_diff_60_w,meanrec_diff_90_360_w,meanrec_diff_30_360_w,meanrec_diff_90_previous_w,meanrec_diff_60_previous_w,meanrec_diff_30_previous_w,meanrec_diff_next_30_w,meanrec_diff_next_60_w,meanrec_diff_before_previous_30_w,meanrec_diff_before_previous_360_w,meanrec_diff_90,meanrec_diff_30,meanrec_diff_60,meanrec_diff_90_360,meanrec_diff_30_360,meanrec_diff_90_previous,meanrec_diff_60_previous,meanrec_diff_30_previous,meanrec_diff_next_30,meanrec_diff_next_60,meanrec_diff_before_previous_30,meanrec_diff_before_previous_360,after_filing_90_focal,before_filing_90_focal,after_filing_30_focal,before_filing_30_focal,after_filing_60_focal,before_filing_60_focal,after_previous_filing_90_focal,after_previous_filing_60_focal,after_previous_filing_30_focal,before_filing_360_focal,after_previous_2year_30_focal,previous_2year_focal,nextyear_30_focal,nextyear_60_focal,after_filing_90_w_focal,before_filing_90_w_focal,after_filing_30_w_focal,before_filing_30_w_focal,after_filing_60_w_focal,before_filing_60_w_focal,after_previous_filing_90_w_focal,after_previous_filing_60_w_focal,after_previous_filing_30_w_focal,before_filing_360_w_focal,after_previous_2year_30_w_focal,previous_2year_w_focal,nextyear_30_w_focal,nextyear_60_w_focal,meanrec_diff_90_focal,meanrec_diff_30_focal,meanrec_diff_60_focal,meanrec_diff_90_360_focal,meanrec_diff_30_360_focal,meanrec_diff_90_previous_focal,meanrec_diff_60_previous_focal,meanrec_diff_30_previous_focal,meanrec_diff_next_30_focal,meanrec_diff_next_60_focal,meanrec_diff_before_previous_30_focal,meanrec_diff_before_previous_360_focal,meanrec_diff_90_w_focal,meanrec_diff_30_w_focal,meanrec_diff_60_w_focal,meanrec_diff_90_360_w_focal,meanrec_diff_30_360_w_focal,meanrec_diff_90_previous_w_focal,meanrec_diff_60_previous_w_focal,meanrec_diff_30_previous_w_focal,meanrec_diff_next_30_w_focal,meanrec_diff_next_60_w_focal,meanrec_diff_before_previous_30_w_focal,meanrec_diff_before_previous_360_w_focal,car1,car3,car5,car7,n_emp_x,log_n_emp_x,hhi_naicsh6_inv_x,hhi_naicsh4_inv_x,hhi_naicsh6_inv_win_x,hhi_naicsh4_inv_win_x,roa_x,roa_win_x,roe_x,roe_win_x,sales_growth_naicsh4_x,sales_growth_naicsh4_win_x,rd_intensity_x,rd_intensity_win_x,naicsh4_x,sale_x,sale_win_x,slack_avail_x,slack_avail_win_x,adv_f_x,adv_f_win_x,intang_f_x,intang_f_win_x,dpt_f_x,dpt_f_win_x,leverage_x,leverage_win_x,tobins_q_x,tobins_q_win_x,market_share_naicsh4_x,prcc_f_x,EPS_x,EPS_win_x,log_at_win_x,at_x,log_at_x,ni_x,rd_f_x,mergers_x,mergers_win_x,market_share_naicsh6_x,market_share_naicsh4_win_x,naicsh6_x,ros_win_x,ros_x,n_emp_y,log_n_emp_y,hhi_naicsh6_inv_y,hhi_naicsh4_inv_y,hhi_naicsh6_inv_win_y,hhi_naicsh4_inv_win_y,roa_y,roa_win_y,roe_y,roe_win_y,sales_growth_naicsh4_y,sales_growth_naicsh4_win_y,rd_intensity_y,rd_intensity_win_y,naicsh4_y,sale_y,sale_win_y,s

----

# Homogeneity with the 2nd Method:
### 1. Calculate the average vector.
### 2. Calculate the cosine similarity between each vector and the average vector.
### 3. Sum all similarities.

### Naicsh4:

In [325]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

embedding_columns = [f'embedding_{i}' for i in range(768)]

# Step 1: Calculate the average vector for each year_modified and naicsh4
def calculate_avg_vectors(df):
    avg_vectors = df.groupby(['year_modified', 'naicsh4'])[embedding_columns].mean().reset_index()
    return avg_vectors

# Step 2: Calculate cosine similarity and sum for each firm within the year_modified and naicsh4
def calculate_and_sum_cosine_similarity(df, avg_vectors):
    # Merge the average vectors with the original dataframe
    merged_df = df.merge(avg_vectors, on=['year_modified', 'naicsh4'], suffixes=('', '_avg'))
    
    # Calculate cosine similarity
    def calculate_similarity(row):
        firm_vector = row[embedding_columns].values.reshape(1, -1)
        avg_vector = row[[f'{col}_avg' for col in embedding_columns]].values.reshape(1, -1)
        return cosine_similarity(firm_vector, avg_vector)[0][0]
    
    merged_df['similarity'] = merged_df.apply(calculate_similarity, axis=1)
    
    # Step 3: Sum all similarities for each year_modified and naicsh4
    similarity_sums = merged_df.groupby(['year_modified', 'naicsh4'])['similarity'].sum().reset_index()
    similarity_sums.rename(columns={'similarity': 'sum_similarity'}, inplace=True)
    
    return similarity_sums

### Naicsh6

In [326]:
embedding_columns = [f'embedding_{i}' for i in range(768)]

# Step 1: Calculate the average vector for each year_modified and naicsh6
def calculate_avg_vectors_6 (df):
    avg_vectors = df.groupby(['year_modified', 'naicsh6'])[embedding_columns].mean().reset_index()
    return avg_vectors

# Step 2: Calculate cosine similarity and sum for each firm within the year_modified and naicsh6
def calculate_and_sum_cosine_similarity_6(df, avg_vectors):
    # Merge the average vectors with the original dataframe
    merged_df = df.merge(avg_vectors, on=['year_modified', 'naicsh6'], suffixes=('', '_avg'))
    
    # Calculate cosine similarity
    def calculate_similarity_6 (row):
        firm_vector = row[embedding_columns].values.reshape(1, -1)
        avg_vector = row[[f'{col}_avg' for col in embedding_columns]].values.reshape(1, -1)
        return cosine_similarity(firm_vector, avg_vector)[0][0]
    
    merged_df['similarity'] = merged_df.apply(calculate_similarity_6, axis=1)
    
    # Step 3: Sum all similarities for each year_modified and naicsh6
    similarity_sums = merged_df.groupby(['year_modified', 'naicsh6'])['similarity'].sum().reset_index()
    similarity_sums.rename(columns={'similarity': 'sum_similarity'}, inplace=True)
    
    return similarity_sums

In [327]:
# Naicsh4
avg_vectors = calculate_avg_vectors(wrds_vector_filter)
result = calculate_and_sum_cosine_similarity(wrds_vector_filter, avg_vectors)
result.rename(columns={'sum_similarity': 'homogeneity_sumsim_naicsh4'}, inplace=True)

# Naicsh6
avg_vectors_6 = calculate_avg_vectors_6 (wrds_vector_filter)
result_6 = calculate_and_sum_cosine_similarity_6 (wrds_vector_filter, avg_vectors_6)
result_6.rename(columns={'sum_similarity': 'homogeneity_sumsim_naicsh6'}, inplace=True)

In [328]:
# Naicsh4
data_acc = pd.merge(data_acc, result, left_on=['year_modified', 'naicsh4 focal'],
                    right_on= ['year_modified', 'naicsh4'], how='left')

data_acc.drop('naicsh4',axis=1, inplace=True)
data_acc.rename(columns={'homogeneity_sumsim_naicsh4': 'homogeneity_sumsim_naicsh4_focal'}, inplace=True)

data_acc = pd.merge(data_acc, result, left_on=['year_modified', 'naicsh4 peer'],
                    right_on= ['year_modified', 'naicsh4'], how='left')

data_acc.drop('naicsh4',axis=1, inplace=True)
data_acc.rename(columns={'homogeneity_sumsim_naicsh4': 'homogeneity_sumsim_naicsh4_peer'}, inplace=True)



# Naicsh6
data_acc = pd.merge(data_acc, result_6, left_on=['year_modified', 'naicsh6 focal'],
                    right_on= ['year_modified', 'naicsh6'], how='left')

data_acc.drop('naicsh6',axis=1, inplace=True)
data_acc.rename(columns={'homogeneity_sumsim_naicsh6': 'homogeneity_sumsim_naicsh6_focal'}, inplace=True)

data_acc = pd.merge(data_acc, result_6, left_on=['year_modified', 'naicsh6 peer'],
                    right_on= ['year_modified', 'naicsh6'], how='left')

data_acc.drop('naicsh6',axis=1, inplace=True)
data_acc.rename(columns={'homogeneity_sumsim_naicsh6': 'homogeneity_sumsim_naicsh6_peer'}, inplace=True)

----

## Normalizing the positive evaluations:

In [331]:
data_acc['upgrade_360_normal']= data_acc['up_after_360']/ data_acc['analysts_peer_number']

data_acc['upgrade_90_normal']= data_acc['up_after_90']/ data_acc['analysts_peer_number']

data_acc['upgrade_360_portion']= data_acc['up_after_360']/ (data_acc['up_after_360'] + data_acc['down_after_360'])

data_acc['upgrade_90_portion']= data_acc['up_after_90']/ (data_acc['up_after_90'] + data_acc['down_after_90'])

-----

# Final Output:

In [333]:
data_acc_sorted = data_acc.sort_values(by=['GVKEY focal', 'year_modified', 'similarity_f0_p0'], ascending=[True, True, False])

data_acc_5 = data_acc_sorted.groupby(['GVKEY focal', 'year_modified']).head(5)

In [3]:
data_acc_sorted = data_acc.sort_values(by=['GVKEY focal', 'year_modified', 'similarity_f0_p0'], ascending=[True, True, False])

data_acc_4 = data_acc_sorted.groupby(['GVKEY focal', 'year_modified']).head(4)

In [3]:
rd4 = data_acc_4.groupby(['GVKEY focal', 'year_modified'])[['rd_intensity_win_y','rd_intensity_y']].mean().reset_index()

rd4.rename (columns={'rd_intensity_win_y': 'rd_4_win', 
                     'rd_intensity_y': 'rd_4'}, inplace=True)

data_acc_4 = pd.merge(data_acc_4, rd4, on=['GVKEY focal', 'year_modified'], how= 'left')

In [4]:
data_acc_6 = data_acc_sorted.groupby(['GVKEY focal', 'year_modified']).head(6)

In [6]:
rd6 = data_acc_6.groupby(['GVKEY focal', 'year_modified'])[['rd_intensity_win_y','rd_intensity_y']].mean().reset_index()

rd6.rename (columns={'rd_intensity_win_y': 'rd_6_win', 
                     'rd_intensity_y': 'rd_6'}, inplace=True)

data_acc_6 = pd.merge(data_acc_6, rd6, on=['GVKEY focal', 'year_modified'], how= 'left')

In [13]:
data_acc_sorted = data_acc.sort_values(by=['GVKEY focal', 'year_modified', 'similarity_f0_p0'], ascending=[True, True, False])

data_acc_7 = data_acc_sorted.groupby(['GVKEY focal', 'year_modified']).head(7)

In [14]:
rd7 = data_acc_7.groupby(['GVKEY focal', 'year_modified'])[['rd_intensity_win_y','rd_intensity_y']].mean().reset_index()

rd7.rename (columns={'rd_intensity_win_y': 'rd_7_win', 
                     'rd_intensity_y': 'rd_7'}, inplace=True)

data_acc_7 = pd.merge(data_acc_7, rd7, on=['GVKEY focal', 'year_modified'], how= 'left')

In [334]:
# Save the file into a CSV format

data_acc.to_csv('data_10R_26May.csv', index=False)

In [335]:
data_acc_5.to_csv('data_5R_26May.csv', index=False)

In [7]:
data_acc_4.to_csv('data_4R_26May.csv', index=False)
data_acc_6.to_csv('data_6R_26May.csv', index=False)

In [16]:
data_acc_7.to_csv('data_7R_26May.csv', index=False)

-----
-----
-----
----

# Correlation matrix with descriptive statistics:

In [17]:
import pandas as pd
import numpy as np

In [18]:
data= pd.read_csv('/.../data_5R_26May.csv')

In [19]:
df_2001= data[(data['year_modified'] >= 2001 ) &(data['year_modified'] <=2021 ) ]

df_2001 ['peer_move_similarity_move_away'] = df_2001['peer_move_similarity'] * df_2001['move_away']

df_2001 ['peer_move_similarity_move_toward'] = df_2001['peer_move_similarity'] * df_2001['move_toward']


/var/folders/8d/hsy71hlj20q70n8qz1r3s4s00000gn/T/ipykernel_5788/1931526553.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2001 ['peer_move_similarity_move_away'] = df_2001['peer_move_similarity'] * df_2001['move_away']
/var/folders/8d/hsy71hlj20q70n8qz1r3s4s00000gn/T/ipykernel_5788/1931526553.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2001 ['peer_move_similarity_move_toward'] = df_2001['peer_move_similarity'] * df_2001['move_toward']


In [20]:

variables = [
    'focal_move_similarity', 'peer_move_similarity_move_away', 'up_after_360' ,  'analysts_peer_number', 'peer_move_similarity_move_toward', 'rd_5_win',
   'common_analysts_focal', 'roa_win_x', 'roa_rel', 'log_at_win_x', 'log_at_rel_win',  'slack_avail_win_x', 'slack_avail_win_y',
  'rd_intensity_win_x', 'rd_intensity_win_y', 'meanrec_diff_90_previous_focal', 'car3', 'strategy_unique_peer', 'strategy_unique_focal',
  'sales_growth_naicsh4_win_x', 'hhi_naicsh4_inv_win_x',
  'similarity_f0_p0', 
  'typicality_4_focal_t1', 'typicality_4_focal_t2', 'avg_change_size_3y_focal', 'moving_toward_industry'
]

# Filter the dataset to include only the specified variables
data_filtered = df_2001[variables]

# Calculate the descriptive statistics
descriptive_stats = data_filtered.agg(['mean', 'std', 'min', 'max']).transpose()
descriptive_stats.columns = ['Mean', 'SD', 'Min', 'Max']

# Calculate the correlation matrix for the filtered data
correlation_matrix = data_filtered.corr().round(2)


In [25]:

# Combine the descriptive statistics with the correlation matrix
# We make sure to only use the lower triangle of the correlation matrix to avoid redundancy
combined_matrix = descriptive_stats.join(correlation_matrix.where(np.tril(np.ones(correlation_matrix.shape), k=-1).astype(bool)))

# Reorder the columns to place 'Mean' and 'SD' at the end
column_order = ['Mean', 'SD', 'Min', 'Max'] + correlation_matrix.columns.tolist() 
combined_matrix = combined_matrix[column_order]

# Reset index to turn the index into a column
combined_matrix.reset_index(inplace=True)
combined_matrix.rename(columns={'index': 'Variable'}, inplace=True)

# Display the combined matrix
combined_matrix.head(len(variables))

,Variable,Mean,SD,Min,Max,focal_move_similarity,peer_move_similarity_move_away,up_after_360,analysts_peer_number,peer_move_similarity_move_toward,...,car3,strategy_unique_peer,strategy_unique_focal,sales_growth_naicsh4_win_x,hhi_naicsh4_inv_win_x,similarity_f0_p0,typicality_4_focal_t1,typicality_4_focal_t2,avg_change_size_3y_focal,moving_toward_industry
0,focal_move_similarity,-0.001679,0.021661,-0.485380,0.477805,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,peer_move_similarity_move_away,-0.006908,0.017423,-0.479508,0.000000,-0.03,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,up_after_360,1.788770,2.246233,0.000000,26.000000,-0.00,-0.01,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,analysts_peer_number,11.181883,8.480483,1.000000,63.000000,0.00,0.03,0.59,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,peer_move_similarity_move_toward,0.001971,0.004658,-0.000000,0.145302,-0.00,0.17,0.01,-0.01,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,rd_5_win,0.382852,1.457604,0.000000,16.552734,0.01,-0.01,-0.07,-0.03,-0.01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,common_analysts_focal,0.202554,0.266582,0.000000,1.000000,0.02,0.06,0.24,0.37,0.04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,roa_win_x,-0.009705,0.236213,-12.165481,0.405738,0.01,0.01,0.03,0.01,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,roa_rel,-0.003880,0.267122,-12.248566,12.365998,0.01,-0.01,-0.06,-0.09,0.01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,log_at_win_x,6.938922,1.796650,0.587787,12.026717,0.00,0.04,0.05,0.09,-0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
combined_matrix = combined_matrix.rename(columns={
    'focal_move_similarity': 'Focal firm repositioning',
    'peer_move_similarity_move_away': 'Rival move away',
    'up_after_360': "Recommendation upgrades of rival's move",
    'analysts_peer_number' : 'Rival analyst coverage',
    'peer_move_similarity_move_toward': 'Rival move toward',
    'rd_5_win' : 'R&D intensity of the environment',
    'common_analysts_focal' : 'Analyst overlap',
    'roa_win_x' : 'Focal firm ROA',
    'roa_rel' : 'Relative ROA',
    'log_at_win_x' : "Focal firm total assets",
    'log_at_rel_win' : 'Relative total assets',
    'slack_avail_win_x' : 'Focal firm slack',
    'slack_avail_win_y' : 'Rival firm slack',
    'rd_intensity_win_x' : 'Focal firm R&D intensity',
    'rd_intensity_win_y' : 'Rival firm R&D intensity',
    'meanrec_diff_90_previous_focal': 'Focal firm mean recom change',
    'similarity_f0_p0' : 'Initial strategic similarity t0',
    'typicality_4_focal_t1': 'Focal firm typicality t1',
    'typicality_4_focal_t2': 'Focal firm typicality t2',
    'moving_toward_industry' : 'Focal firm movement toward trends',
    'car3' : 'Rival firm abnormal return',
    'strategy_unique_peer' : 'Rival strategy uniqueness',
    'strategy_unique_focal' : 'Focal firm strategy uniqueness',
    'sales_growth_naicsh4_win_x' : 'Focal firm industry sales growth',
    'hhi_naicsh4_inv_win_x' : 'Focal firm industry competition (1-hhi)',
    'avg_change_size_3y_focal' : 'Focal firm change size average in the past three years'
    
})

In [27]:
combined_matrix.to_excel("CorrMatrix_29052024.xlsx", sheet_name='Sheet1', index=False)

-----
-----
-----
----